# 7th_2018

## Raw Data

In [1]:
import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)

    Example:
    --------
    blob_url = "https://github.com/user/repo/blob/main/file.xls"
    raw_url = convert_github_url_to_raw(blob_url)
    # "https://github.com/user/repo/raw/main/file.xls"
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_7th_governor_election(file_path_or_url):
    """
    제7회 시도지사 선거 데이터를 처리하는 함수

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL

    Returns:
    --------
    dict
        각 선거구별로 첫 번째 행, '시도명'이 '합계'인 행, '읍면동명'이 '계'인 행만 포함한 데이터프레임 딕셔너리

    Example:
    --------
    # 로컬 파일
    filtered_dfs = process_7th_governor_election('201806197지선01시도지사_읍면동별개표자료.xlsx')

    # GitHub URL
    github_url = "https://github.com/.../blob/main/201806197지선01시도지사_읍면동별개표자료.xlsx"
    filtered_dfs = process_7th_governor_election(github_url)
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 첫 번째 행을 헤더로 사용하여 파일 읽기
    df = pd.read_excel(file_path_or_url)

    print("데이터프레임 열 이름:")
    print(df.columns.tolist()[:10])  # 처음 10개만 출력
    print(f"\n총 {len(df.columns)}개의 열")

    # '선거구명' 열의 고유값 확인
    if '선거구명' not in df.columns:
        print("경고: '선거구명' 열이 없습니다. 첫 번째 열을 사용합니다.")
        election_column = df.columns[0]
    else:
        election_column = '선거구명'

    unique_values = df[election_column].unique()
    print(f"\n'{election_column}' 열의 고유값들:")
    print(unique_values)
    print(f"\n총 {len(unique_values)}개의 그룹으로 분리됩니다.\n")

    # 딕셔너리로 각 그룹별 데이터프레임 저장
    grouped_dfs = {}

    # 선거구명별로 데이터프레임 분리
    for value in unique_values:
        # 해당 값과 일치하는 행들만 필터링
        grouped_dfs[value] = df[df[election_column] == value].copy()
        print(f"'{value}' 그룹: {len(grouped_dfs[value])}개 행")

    # 각 데이터프레임에서 필요한 행만 필터링
    filtered_dfs = {}

    for value, group_df in grouped_dfs.items():
        # 빈 데이터프레임 체크
        if len(group_df) == 0:
            print(f"'{value}' 그룹은 비어있습니다. 건너뜁니다.")
            continue

        # 필터링할 행들을 담을 리스트
        rows_to_keep = []

        # 첫 번째 행 추가
        rows_to_keep.append(group_df.iloc[0])

        # '시도명'이 '합계'인 행 찾기
        if '시도명' in group_df.columns:
            summary_rows_sido = group_df[group_df['시도명'] == '합계']
            if not summary_rows_sido.empty:
                for idx, row in summary_rows_sido.iterrows():
                    rows_to_keep.append(row)

        # '읍면동명'이 '계'인 행 찾기
        if '읍면동명' in group_df.columns:
            summary_rows_dong = group_df[group_df['읍면동명'] == '계']
            if not summary_rows_dong.empty:
                for idx, row in summary_rows_dong.iterrows():
                    rows_to_keep.append(row)

        # 데이터프레임으로 변환 (중복 제거)
        if len(rows_to_keep) > 1:
            filtered_df = pd.DataFrame(rows_to_keep).drop_duplicates()
        else:
            filtered_df = pd.DataFrame([rows_to_keep[0]])

        filtered_dfs[value] = filtered_df.reset_index(drop=True)
        print(f"'{value}' 그룹: 원본 {len(group_df)}개 행 → 필터링 후 {len(filtered_df)}개 행")

    print(f"\n처리 완료! 총 {len(filtered_dfs)}개의 선거구별 데이터프레임이 생성되었습니다.")

    return filtered_dfs


# 사용 예시

# 7회 지방선거 (2018년)
# github_url = "https://github.com/user/korean-elections/blob/main/original/Local_Elections_Governor/7th_2018/201806197지선01시도지사_읍면동별개표자료.xlsx"
# filtered_dfs7 = process_7th_governor_election(github_url)

# 결과 확인
# print("\n사용 가능한 선거구:")
# print(list(filtered_dfs7.keys()))

# 특정 선거구 데이터 확인
# print("\n서울특별시 데이터:")
# print(filtered_dfs7['서울특별시'])

In [2]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url7 = "https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/7th_2018/20180619-7%EC%A7%80%EC%84%A0-01-(%EC%8B%9C%EB%8F%84%EC%A7%80%EC%82%AC)_%EC%9D%8D%EB%A9%B4%EB%8F%99%EB%B3%84%EA%B0%9C%ED%91%9C%EC%9E%90%EB%A3%8C.xlsx"
dfs_7th = process_7th_governor_election(blob_url7)

데이터프레임 열 이름:
['선거종류', '선거구명', '시도명', '구시군명', '읍면동명', '구분', '선거인수', '투표수', '후보자별 득표수', 'Unnamed: 9']

총 20개의 열

'선거구명' 열의 고유값들:
['서울특별시' '부산광역시' '대구광역시' '인천광역시' '광주광역시' '대전광역시' '울산광역시' '세종특별자치시' '경기도'
 '강원도' '충청북도' '충청남도' '전라북도' '전라남도' '경상북도' '경상남도' '제주특별자치도']

총 17개의 그룹으로 분리됩니다.

'서울특별시' 그룹: 1371개 행
'부산광역시' 그룹: 684개 행
'대구광역시' 그룹: 451개 행
'인천광역시' 그룹: 495개 행
'광주광역시' 그룹: 307개 행
'대전광역시' 그룹: 259개 행
'울산광역시' 그룹: 190개 행
'세종특별자치시' 그룹: 57개 행
'경기도' 그룹: 1853개 행
'강원도' 그룹: 638개 행
'충청북도' 그룹: 517개 행
'충청남도' 그룹: 687개 행
'전라북도' 그룹: 785개 행
'전라남도' 그룹: 981개 행
'경상북도' 그룹: 1094개 행
'경상남도' 그룹: 1014개 행
'제주특별자치도' 그룹: 139개 행
'서울특별시' 그룹: 원본 1371개 행 → 필터링 후 27개 행
'부산광역시' 그룹: 원본 684개 행 → 필터링 후 18개 행
'대구광역시' 그룹: 원본 451개 행 → 필터링 후 10개 행
'인천광역시' 그룹: 원본 495개 행 → 필터링 후 12개 행
'광주광역시' 그룹: 원본 307개 행 → 필터링 후 7개 행
'대전광역시' 그룹: 원본 259개 행 → 필터링 후 7개 행
'울산광역시' 그룹: 원본 190개 행 → 필터링 후 7개 행
'세종특별자치시' 그룹: 원본 57개 행 → 필터링 후 3개 행
'경기도' 그룹: 원본 1853개 행 → 필터링 후 44개 행
'강원도' 그룹: 원본 638개 행 → 필터링 후 20개 행
'충청북도' 그룹: 원본 517개 행 → 필터링 후 16개 행
'충청남도' 그룹

In [3]:
list(dfs_7th.keys())

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

## Seoul


In [4]:
seoul_7th = dfs_7th['서울특별시']
seoul_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n박원순,자유한국당_x000D_\n김문수,바른미래당_x000D_\n안철수,정의당_x000D_\n김종민,민중당_x000D_\n김진숙,대한애국당_x000D_\n인지연,녹색당_x000D_\n신지예,우리미래_x000D_\n우인철,친박연대_x000D_\n최태현,계,NaN,NaN
1,NaN,서울특별시,합계,NaN,NaN,NaN,8380947.0,5019098.0,2619497,1158487,970374,81664,22134,11222,82874,11599,4021,4961872,57226.0,3361849.0
2,NaN,서울특별시,서울특별시,종로구,계,NaN,134963.0,81195.0,41148,18777,15946,1469,358,223,1896,215,59,80091,1104.0,53768.0
3,NaN,서울특별시,서울특별시,중구,계,NaN,112336.0,66474.0,34243,15618,12814,1023,279,193,1076,149,96,65491,983.0,45862.0
4,NaN,서울특별시,서울특별시,용산구,계,NaN,200009.0,116054.0,53964,31073,24074,2151,436,231,2392,226,115,114662,1392.0,83955.0
5,NaN,서울특별시,서울특별시,성동구,계,NaN,264634.0,157863.0,84106,34878,30379,2594,689,309,2418,370,123,155866,1997.0,106771.0
6,NaN,서울특별시,서울특별시,광진구,계,NaN,308174.0,180373.0,99366,38606,32310,3077,804,376,3222,482,125,178368,2005.0,127801.0
7,NaN,서울특별시,서울특별시,동대문구,계,NaN,302939.0,178077.0,93913,40984,33437,2731,827,434,2848,404,145,175723,2354.0,124862.0
8,NaN,서울특별시,서울특별시,중랑구,계,NaN,352041.0,198943.0,109318,45905,33672,2662,959,519,2561,429,185,196210,2733.0,153098.0
9,NaN,서울특별시,서울특별시,성북구,계,NaN,373174.0,222184.0,119898,47274,42105,3750,918,523,4349,497,187,219501,2683.0,150990.0


In [5]:
seoul_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [6]:
rename_seoul = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_박원순',
    'Unnamed: 9': '득표수_2_자유한국당_김문수',
    'Unnamed: 10': '득표수_3_바른미래당_안철수',
    'Unnamed: 11': '득표수_5_정의당_김종민',
    'Unnamed: 12': '득표수_6_민중당_김진숙',
    'Unnamed: 13': '득표수_7_대한애국당_인지연',
    'Unnamed: 14': '득표수_8_녹색당_신지예',
    'Unnamed: 15': '득표수_9_미래당_우인철',
    'Unnamed: 16': '득표수_10_친박연대_최태현',
    'Unnamed: 17': '득표수_계'
}

In [7]:
seoul_7th = seoul_7th.rename(columns=rename_seoul).drop(columns=['선거종류', '선거구명', '읍면동명', '구분']).drop(index=0)
seoul_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박원순,득표수_2_자유한국당_김문수,득표수_3_바른미래당_안철수,득표수_5_정의당_김종민,득표수_6_민중당_김진숙,득표수_7_대한애국당_인지연,득표수_8_녹색당_신지예,득표수_9_미래당_우인철,득표수_10_친박연대_최태현,득표수_계,무효투표수,기권수
1,합계,NaN,8380947.0,5019098.0,2619497,1158487,970374,81664,22134,11222,82874,11599,4021,4961872,57226.0,3361849.0
2,서울특별시,종로구,134963.0,81195.0,41148,18777,15946,1469,358,223,1896,215,59,80091,1104.0,53768.0
3,서울특별시,중구,112336.0,66474.0,34243,15618,12814,1023,279,193,1076,149,96,65491,983.0,45862.0
4,서울특별시,용산구,200009.0,116054.0,53964,31073,24074,2151,436,231,2392,226,115,114662,1392.0,83955.0
5,서울특별시,성동구,264634.0,157863.0,84106,34878,30379,2594,689,309,2418,370,123,155866,1997.0,106771.0
6,서울특별시,광진구,308174.0,180373.0,99366,38606,32310,3077,804,376,3222,482,125,178368,2005.0,127801.0
7,서울특별시,동대문구,302939.0,178077.0,93913,40984,33437,2731,827,434,2848,404,145,175723,2354.0,124862.0
8,서울특별시,중랑구,352041.0,198943.0,109318,45905,33672,2662,959,519,2561,429,185,196210,2733.0,153098.0
9,서울특별시,성북구,373174.0,222184.0,119898,47274,42105,3750,918,523,4349,497,187,219501,2683.0,150990.0
10,서울특별시,강북구,280860.0,156279.0,85839,33537,28138,2396,718,481,2276,431,165,153981,2298.0,124581.0


In [8]:
seoul_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 1 to 26
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                26 non-null     object 
 1   구시군               25 non-null     object 
 2   선거인수              26 non-null     float64
 3   투표수               26 non-null     float64
 4   득표수_1_더불어민주당_박원순  26 non-null     object 
 5   득표수_2_자유한국당_김문수   26 non-null     object 
 6   득표수_3_바른미래당_안철수   26 non-null     object 
 7   득표수_5_정의당_김종민     26 non-null     object 
 8   득표수_6_민중당_김진숙     26 non-null     object 
 9   득표수_7_대한애국당_인지연   26 non-null     object 
 10  득표수_8_녹색당_신지예     26 non-null     object 
 11  득표수_9_미래당_우인철     26 non-null     object 
 12  득표수_10_친박연대_최태현   26 non-null     object 
 13  득표수_계             26 non-null     object 
 14  무효투표수             26 non-null     float64
 15  기권수               26 non-null     float64
dtypes: float64(4), object(12)
memory usage: 3.4+ K

In [9]:
seoul_7th = seoul_7th.astype({col: int for col in seoul_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
seoul_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박원순,득표수_2_자유한국당_김문수,득표수_3_바른미래당_안철수,득표수_5_정의당_김종민,득표수_6_민중당_김진숙,득표수_7_대한애국당_인지연,득표수_8_녹색당_신지예,득표수_9_미래당_우인철,득표수_10_친박연대_최태현,득표수_계,무효투표수,기권수
1,합계,NaN,8380947,5019098,2619497,1158487,970374,81664,22134,11222,82874,11599,4021,4961872,57226,3361849
2,서울특별시,종로구,134963,81195,41148,18777,15946,1469,358,223,1896,215,59,80091,1104,53768
3,서울특별시,중구,112336,66474,34243,15618,12814,1023,279,193,1076,149,96,65491,983,45862
4,서울특별시,용산구,200009,116054,53964,31073,24074,2151,436,231,2392,226,115,114662,1392,83955
5,서울특별시,성동구,264634,157863,84106,34878,30379,2594,689,309,2418,370,123,155866,1997,106771
6,서울특별시,광진구,308174,180373,99366,38606,32310,3077,804,376,3222,482,125,178368,2005,127801
7,서울특별시,동대문구,302939,178077,93913,40984,33437,2731,827,434,2848,404,145,175723,2354,124862
8,서울특별시,중랑구,352041,198943,109318,45905,33672,2662,959,519,2561,429,185,196210,2733,153098
9,서울특별시,성북구,373174,222184,119898,47274,42105,3750,918,523,4349,497,187,219501,2683,150990
10,서울특별시,강북구,280860,156279,85839,33537,28138,2396,718,481,2276,431,165,153981,2298,124581


In [10]:
seoul_7th_with_total = seoul_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '서울특별시'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [11]:
seoul_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박원순,득표수_2_자유한국당_김문수,득표수_3_바른미래당_안철수,득표수_5_정의당_김종민,득표수_6_민중당_김진숙,득표수_7_대한애국당_인지연,득표수_8_녹색당_신지예,득표수_9_미래당_우인철,득표수_10_친박연대_최태현,득표수_계,무효투표수,기권수
1,서울특별시,합계,8380947,5019098,2619497,1158487,970374,81664,22134,11222,82874,11599,4021,4961872,57226,3361849
2,서울특별시,종로구,134963,81195,41148,18777,15946,1469,358,223,1896,215,59,80091,1104,53768
3,서울특별시,중구,112336,66474,34243,15618,12814,1023,279,193,1076,149,96,65491,983,45862
4,서울특별시,용산구,200009,116054,53964,31073,24074,2151,436,231,2392,226,115,114662,1392,83955
5,서울특별시,성동구,264634,157863,84106,34878,30379,2594,689,309,2418,370,123,155866,1997,106771
6,서울특별시,광진구,308174,180373,99366,38606,32310,3077,804,376,3222,482,125,178368,2005,127801
7,서울특별시,동대문구,302939,178077,93913,40984,33437,2731,827,434,2848,404,145,175723,2354,124862
8,서울특별시,중랑구,352041,198943,109318,45905,33672,2662,959,519,2561,429,185,196210,2733,153098
9,서울특별시,성북구,373174,222184,119898,47274,42105,3750,918,523,4349,497,187,219501,2683,150990
10,서울특별시,강북구,280860,156279,85839,33537,28138,2396,718,481,2276,431,165,153981,2298,124581


In [12]:
seoul_7th_with_total.to_csv("temp1_governor_seoul_7.csv", index=False, encoding="utf-8-sig")


## Busan


In [13]:
busan_7th = dfs_7th['부산광역시']
busan_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,부산광역시,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n오거돈,자유한국당_x000D_\n서병수,바른미래당_x000D_\n이성권,정의당_x000D_\n박주미,무소속_x000D_\n이종혁,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,부산광역시,합계,NaN,NaN,NaN,2939046.0,1727684.0,940469,632806,67502,35299,26720,0,0,0,0,1702796,24888.0,1211362.0
2,NaN,부산광역시,부산광역시,중구,계,NaN,39832.0,22499.0,11730,8993,657,423,300,0,0,0,0,22103,396.0,17333.0
3,NaN,부산광역시,부산광역시,서구,계,NaN,97123.0,55377.0,27919,22760,1670,1091,913,0,0,0,0,54353,1024.0,41746.0
4,NaN,부산광역시,부산광역시,동구,계,NaN,79091.0,46288.0,23476,18841,1480,833,691,0,0,0,0,45321,967.0,32803.0
5,NaN,부산광역시,부산광역시,영도구,계,NaN,107930.0,61332.0,34279,22179,1923,1050,879,0,0,0,0,60310,1022.0,46598.0
6,NaN,부산광역시,부산광역시,부산진구,계,NaN,319402.0,184063.0,98314,67354,8409,3608,3626,0,0,0,0,181311,2752.0,135339.0
7,NaN,부산광역시,부산광역시,동래구,계,NaN,226713.0,135595.0,72895,50115,6046,2847,1963,0,0,0,0,133866,1729.0,91118.0
8,NaN,부산광역시,부산광역시,남구,계,NaN,234211.0,140451.0,75978,52232,5283,2980,1902,0,0,0,0,138375,2076.0,93760.0
9,NaN,부산광역시,부산광역시,북구,계,NaN,253907.0,152125.0,85791,53265,5742,3296,2034,0,0,0,0,150128,1997.0,101782.0


In [14]:
busan_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [15]:
rename_busan = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_오거돈',
    'Unnamed: 9': '득표수_2_자유한국당_서병수',
    'Unnamed: 10': '득표수_3_바른미래당_이성권',
    'Unnamed: 11': '득표수_5_정의당_박주미',
    'Unnamed: 12': '득표수_6_무소속_이종혁',
    'Unnamed: 17': '득표수_계'
}

In [16]:
busan_7th = busan_7th.rename(columns=rename_busan).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
busan_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오거돈,득표수_2_자유한국당_서병수,득표수_3_바른미래당_이성권,득표수_5_정의당_박주미,득표수_6_무소속_이종혁,득표수_계,무효투표수,기권수
1,합계,NaN,2939046.0,1727684.0,940469,632806,67502,35299,26720,1702796,24888.0,1211362.0
2,부산광역시,중구,39832.0,22499.0,11730,8993,657,423,300,22103,396.0,17333.0
3,부산광역시,서구,97123.0,55377.0,27919,22760,1670,1091,913,54353,1024.0,41746.0
4,부산광역시,동구,79091.0,46288.0,23476,18841,1480,833,691,45321,967.0,32803.0
5,부산광역시,영도구,107930.0,61332.0,34279,22179,1923,1050,879,60310,1022.0,46598.0
6,부산광역시,부산진구,319402.0,184063.0,98314,67354,8409,3608,3626,181311,2752.0,135339.0
7,부산광역시,동래구,226713.0,135595.0,72895,50115,6046,2847,1963,133866,1729.0,91118.0
8,부산광역시,남구,234211.0,140451.0,75978,52232,5283,2980,1902,138375,2076.0,93760.0
9,부산광역시,북구,253907.0,152125.0,85791,53265,5742,3296,2034,150128,1997.0,101782.0
10,부산광역시,해운대구,342079.0,200332.0,109622,73131,8980,3671,2770,198174,2158.0,141747.0


In [17]:
busan_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 1 to 17
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                17 non-null     object 
 1   구시군               16 non-null     object 
 2   선거인수              17 non-null     float64
 3   투표수               17 non-null     float64
 4   득표수_1_더불어민주당_오거돈  17 non-null     object 
 5   득표수_2_자유한국당_서병수   17 non-null     object 
 6   득표수_3_바른미래당_이성권   17 non-null     object 
 7   득표수_5_정의당_박주미     17 non-null     object 
 8   득표수_6_무소속_이종혁     17 non-null     object 
 9   득표수_계             17 non-null     object 
 10  무효투표수             17 non-null     float64
 11  기권수               17 non-null     float64
dtypes: float64(4), object(8)
memory usage: 1.7+ KB


In [18]:
busan_7th = busan_7th.astype({col: int for col in busan_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
busan_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오거돈,득표수_2_자유한국당_서병수,득표수_3_바른미래당_이성권,득표수_5_정의당_박주미,득표수_6_무소속_이종혁,득표수_계,무효투표수,기권수
1,합계,NaN,2939046,1727684,940469,632806,67502,35299,26720,1702796,24888,1211362
2,부산광역시,중구,39832,22499,11730,8993,657,423,300,22103,396,17333
3,부산광역시,서구,97123,55377,27919,22760,1670,1091,913,54353,1024,41746
4,부산광역시,동구,79091,46288,23476,18841,1480,833,691,45321,967,32803
5,부산광역시,영도구,107930,61332,34279,22179,1923,1050,879,60310,1022,46598
6,부산광역시,부산진구,319402,184063,98314,67354,8409,3608,3626,181311,2752,135339
7,부산광역시,동래구,226713,135595,72895,50115,6046,2847,1963,133866,1729,91118
8,부산광역시,남구,234211,140451,75978,52232,5283,2980,1902,138375,2076,93760
9,부산광역시,북구,253907,152125,85791,53265,5742,3296,2034,150128,1997,101782
10,부산광역시,해운대구,342079,200332,109622,73131,8980,3671,2770,198174,2158,141747


In [19]:
busan_7th_with_total = busan_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '부산광역시'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [20]:
busan_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오거돈,득표수_2_자유한국당_서병수,득표수_3_바른미래당_이성권,득표수_5_정의당_박주미,득표수_6_무소속_이종혁,득표수_계,무효투표수,기권수
1,부산광역시,합계,2939046,1727684,940469,632806,67502,35299,26720,1702796,24888,1211362
2,부산광역시,중구,39832,22499,11730,8993,657,423,300,22103,396,17333
3,부산광역시,서구,97123,55377,27919,22760,1670,1091,913,54353,1024,41746
4,부산광역시,동구,79091,46288,23476,18841,1480,833,691,45321,967,32803
5,부산광역시,영도구,107930,61332,34279,22179,1923,1050,879,60310,1022,46598
6,부산광역시,부산진구,319402,184063,98314,67354,8409,3608,3626,181311,2752,135339
7,부산광역시,동래구,226713,135595,72895,50115,6046,2847,1963,133866,1729,91118
8,부산광역시,남구,234211,140451,75978,52232,5283,2980,1902,138375,2076,93760
9,부산광역시,북구,253907,152125,85791,53265,5742,3296,2034,150128,1997,101782
10,부산광역시,해운대구,342079,200332,109622,73131,8980,3671,2770,198174,2158,141747


In [21]:
busan_7th_with_total.to_csv("temp1_governor_busan_7.csv", index=False, encoding="utf-8-sig")


## Daegu


In [22]:
daegu_7th = dfs_7th['대구광역시']
daegu_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,대구광역시,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n임대윤,자유한국당_x000D_\n권영진,바른미래당_x000D_\n김형기,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,대구광역시,합계,NaN,NaN,NaN,2047286.0,1172279.0,458112,619165,74955,0,0,0,0,0,0,1152232,20047.0,875007.0
2,NaN,대구광역시,대구광역시,중구,계,NaN,68303.0,39863.0,14983,21640,2572,0,0,0,0,0,0,39195,668.0,28440.0
3,NaN,대구광역시,대구광역시,동구,계,NaN,295860.0,171627.0,65465,90378,12894,0,0,0,0,0,0,168737,2890.0,124233.0
4,NaN,대구광역시,대구광역시,서구,계,NaN,167260.0,89634.0,28180,53879,5532,0,0,0,0,0,0,87591,2043.0,77626.0
5,NaN,대구광역시,대구광역시,남구,계,NaN,134493.0,72239.0,25478,41302,3886,0,0,0,0,0,0,70666,1573.0,62254.0
6,NaN,대구광역시,대구광역시,북구,계,NaN,360010.0,205858.0,83084,106028,13244,0,0,0,0,0,0,202356,3502.0,154152.0
7,NaN,대구광역시,대구광역시,수성구,계,NaN,351854.0,211819.0,84824,111722,12635,0,0,0,0,0,0,209181,2638.0,140035.0
8,NaN,대구광역시,대구광역시,달서구,계,NaN,472669.0,266010.0,108602,136139,16996,0,0,0,0,0,0,261737,4273.0,206659.0
9,NaN,대구광역시,대구광역시,달성군,계,NaN,196837.0,115229.0,47496,58077,7196,0,0,0,0,0,0,112769,2460.0,81608.0


In [23]:
daegu_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [24]:
rename_daegu = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_임대윤',
    'Unnamed: 9': '득표수_2_자유한국당_권영진',
    'Unnamed: 10': '득표수_3_바른미래당_김형기',
    'Unnamed: 17': '득표수_계'
}

In [25]:
daegu_7th = daegu_7th.rename(columns=rename_daegu).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
daegu_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_임대윤,득표수_2_자유한국당_권영진,득표수_3_바른미래당_김형기,득표수_계,무효투표수,기권수
1,합계,NaN,2047286.0,1172279.0,458112,619165,74955,1152232,20047.0,875007.0
2,대구광역시,중구,68303.0,39863.0,14983,21640,2572,39195,668.0,28440.0
3,대구광역시,동구,295860.0,171627.0,65465,90378,12894,168737,2890.0,124233.0
4,대구광역시,서구,167260.0,89634.0,28180,53879,5532,87591,2043.0,77626.0
5,대구광역시,남구,134493.0,72239.0,25478,41302,3886,70666,1573.0,62254.0
6,대구광역시,북구,360010.0,205858.0,83084,106028,13244,202356,3502.0,154152.0
7,대구광역시,수성구,351854.0,211819.0,84824,111722,12635,209181,2638.0,140035.0
8,대구광역시,달서구,472669.0,266010.0,108602,136139,16996,261737,4273.0,206659.0
9,대구광역시,달성군,196837.0,115229.0,47496,58077,7196,112769,2460.0,81608.0


In [26]:
daegu_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 1 to 9
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                9 non-null      object 
 1   구시군               8 non-null      object 
 2   선거인수              9 non-null      float64
 3   투표수               9 non-null      float64
 4   득표수_1_더불어민주당_임대윤  9 non-null      object 
 5   득표수_2_자유한국당_권영진   9 non-null      object 
 6   득표수_3_바른미래당_김형기   9 non-null      object 
 7   득표수_계             9 non-null      object 
 8   무효투표수             9 non-null      float64
 9   기권수               9 non-null      float64
dtypes: float64(4), object(6)
memory usage: 852.0+ bytes


In [27]:
daegu_7th = daegu_7th.astype({col: int for col in daegu_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
daegu_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_임대윤,득표수_2_자유한국당_권영진,득표수_3_바른미래당_김형기,득표수_계,무효투표수,기권수
1,합계,NaN,2047286,1172279,458112,619165,74955,1152232,20047,875007
2,대구광역시,중구,68303,39863,14983,21640,2572,39195,668,28440
3,대구광역시,동구,295860,171627,65465,90378,12894,168737,2890,124233
4,대구광역시,서구,167260,89634,28180,53879,5532,87591,2043,77626
5,대구광역시,남구,134493,72239,25478,41302,3886,70666,1573,62254
6,대구광역시,북구,360010,205858,83084,106028,13244,202356,3502,154152
7,대구광역시,수성구,351854,211819,84824,111722,12635,209181,2638,140035
8,대구광역시,달서구,472669,266010,108602,136139,16996,261737,4273,206659
9,대구광역시,달성군,196837,115229,47496,58077,7196,112769,2460,81608


In [28]:
daegu_7th_with_total = daegu_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '대구광역시'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)


In [29]:
daegu_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_임대윤,득표수_2_자유한국당_권영진,득표수_3_바른미래당_김형기,득표수_계,무효투표수,기권수
1,대구광역시,합계,2047286,1172279,458112,619165,74955,1152232,20047,875007
2,대구광역시,중구,68303,39863,14983,21640,2572,39195,668,28440
3,대구광역시,동구,295860,171627,65465,90378,12894,168737,2890,124233
4,대구광역시,서구,167260,89634,28180,53879,5532,87591,2043,77626
5,대구광역시,남구,134493,72239,25478,41302,3886,70666,1573,62254
6,대구광역시,북구,360010,205858,83084,106028,13244,202356,3502,154152
7,대구광역시,수성구,351854,211819,84824,111722,12635,209181,2638,140035
8,대구광역시,달서구,472669,266010,108602,136139,16996,261737,4273,206659
9,대구광역시,달성군,196837,115229,47496,58077,7196,112769,2460,81608


In [30]:
daegu_7th_with_total.to_csv("temp1_governor_daegu_7.csv", index=False, encoding="utf-8-sig")


## Incheon


In [31]:
incheon_7th = dfs_7th['인천광역시']
incheon_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,인천광역시,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n박남춘,자유한국당_x000D_\n유정복,바른미래당_x000D_\n문병호,정의당_x000D_\n김응호,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,인천광역시,합계,NaN,NaN,NaN,2440779.0,1349084.0,766186,470937,54054,37472,0,0,0,0,0,1328649,20435.0,1091695.0
2,NaN,인천광역시,인천광역시,중구,계,NaN,98608.0,53666.0,29817,19724,1960,1301,0,0,0,0,0,52802,864.0,44942.0
3,NaN,인천광역시,인천광역시,동구,계,NaN,57559.0,34757.0,17288,14655,895,1175,0,0,0,0,0,34013,744.0,22802.0
4,NaN,인천광역시,인천광역시,남구,계,NaN,356508.0,183835.0,97347,71278,6754,5318,0,0,0,0,0,180697,3138.0,172673.0
5,NaN,인천광역시,인천광역시,연수구,계,NaN,269626.0,160548.0,90969,58191,5277,4153,0,0,0,0,0,158590,1958.0,109078.0
6,NaN,인천광역시,인천광역시,남동구,계,NaN,441132.0,246564.0,144979,82245,8552,7488,0,0,0,0,0,243264,3300.0,194568.0
7,NaN,인천광역시,인천광역시,부평구,계,NaN,448618.0,241811.0,140735,76655,13830,7186,0,0,0,0,0,238406,3405.0,206807.0
8,NaN,인천광역시,인천광역시,계양구,계,NaN,266216.0,143889.0,86747,44686,5869,4525,0,0,0,0,0,141827,2062.0,122327.0
9,NaN,인천광역시,인천광역시,서구,계,NaN,422819.0,230166.0,137825,74876,9347,5215,0,0,0,0,0,227263,2903.0,192653.0


In [32]:
incheon_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [33]:
rename_incheon = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_박남춘',
    'Unnamed: 9': '득표수_2_자유한국당_유정복',
    'Unnamed: 10': '득표수_3_바른미래당_문병호',
    'Unnamed: 11': '득표수_5_정의당_김응호',
    'Unnamed: 17': '득표수_계'
}

In [34]:
incheon_7th = incheon_7th.rename(columns=rename_incheon).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
incheon_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박남춘,득표수_2_자유한국당_유정복,득표수_3_바른미래당_문병호,득표수_5_정의당_김응호,득표수_계,무효투표수,기권수
1,합계,NaN,2440779.0,1349084.0,766186,470937,54054,37472,1328649,20435.0,1091695.0
2,인천광역시,중구,98608.0,53666.0,29817,19724,1960,1301,52802,864.0,44942.0
3,인천광역시,동구,57559.0,34757.0,17288,14655,895,1175,34013,744.0,22802.0
4,인천광역시,남구,356508.0,183835.0,97347,71278,6754,5318,180697,3138.0,172673.0
5,인천광역시,연수구,269626.0,160548.0,90969,58191,5277,4153,158590,1958.0,109078.0
6,인천광역시,남동구,441132.0,246564.0,144979,82245,8552,7488,243264,3300.0,194568.0
7,인천광역시,부평구,448618.0,241811.0,140735,76655,13830,7186,238406,3405.0,206807.0
8,인천광역시,계양구,266216.0,143889.0,86747,44686,5869,4525,141827,2062.0,122327.0
9,인천광역시,서구,422819.0,230166.0,137825,74876,9347,5215,227263,2903.0,192653.0
10,인천광역시,강화군,60755.0,39940.0,14972,21723,1080,757,38532,1408.0,20815.0


In [35]:
incheon_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 1 to 11
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                11 non-null     object 
 1   구시군               10 non-null     object 
 2   선거인수              11 non-null     float64
 3   투표수               11 non-null     float64
 4   득표수_1_더불어민주당_박남춘  11 non-null     object 
 5   득표수_2_자유한국당_유정복   11 non-null     object 
 6   득표수_3_바른미래당_문병호   11 non-null     object 
 7   득표수_5_정의당_김응호     11 non-null     object 
 8   득표수_계             11 non-null     object 
 9   무효투표수             11 non-null     float64
 10  기권수               11 non-null     float64
dtypes: float64(4), object(7)
memory usage: 1.1+ KB


In [36]:
incheon_7th = incheon_7th.astype({col: int for col in incheon_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
incheon_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박남춘,득표수_2_자유한국당_유정복,득표수_3_바른미래당_문병호,득표수_5_정의당_김응호,득표수_계,무효투표수,기권수
1,합계,NaN,2440779,1349084,766186,470937,54054,37472,1328649,20435,1091695
2,인천광역시,중구,98608,53666,29817,19724,1960,1301,52802,864,44942
3,인천광역시,동구,57559,34757,17288,14655,895,1175,34013,744,22802
4,인천광역시,남구,356508,183835,97347,71278,6754,5318,180697,3138,172673
5,인천광역시,연수구,269626,160548,90969,58191,5277,4153,158590,1958,109078
6,인천광역시,남동구,441132,246564,144979,82245,8552,7488,243264,3300,194568
7,인천광역시,부평구,448618,241811,140735,76655,13830,7186,238406,3405,206807
8,인천광역시,계양구,266216,143889,86747,44686,5869,4525,141827,2062,122327
9,인천광역시,서구,422819,230166,137825,74876,9347,5215,227263,2903,192653
10,인천광역시,강화군,60755,39940,14972,21723,1080,757,38532,1408,20815


In [37]:
incheon_7th_with_total = incheon_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '인천광역시'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [38]:
incheon_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박남춘,득표수_2_자유한국당_유정복,득표수_3_바른미래당_문병호,득표수_5_정의당_김응호,득표수_계,무효투표수,기권수
1,인천광역시,합계,2440779,1349084,766186,470937,54054,37472,1328649,20435,1091695
2,인천광역시,중구,98608,53666,29817,19724,1960,1301,52802,864,44942
3,인천광역시,동구,57559,34757,17288,14655,895,1175,34013,744,22802
4,인천광역시,남구,356508,183835,97347,71278,6754,5318,180697,3138,172673
5,인천광역시,연수구,269626,160548,90969,58191,5277,4153,158590,1958,109078
6,인천광역시,남동구,441132,246564,144979,82245,8552,7488,243264,3300,194568
7,인천광역시,부평구,448618,241811,140735,76655,13830,7186,238406,3405,206807
8,인천광역시,계양구,266216,143889,86747,44686,5869,4525,141827,2062,122327
9,인천광역시,서구,422819,230166,137825,74876,9347,5215,227263,2903,192653
10,인천광역시,강화군,60755,39940,14972,21723,1080,757,38532,1408,20815


In [39]:
incheon_7th_with_total.to_csv("temp1_governor_incheon_7.csv", index=False, encoding="utf-8-sig")


## Gwangju


In [40]:
gwangju_7th = dfs_7th['광주광역시']
gwangju_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,광주광역시,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n이용섭,바른미래당_x000D_\n전덕영,정의당_x000D_\n나경채,민중당_x000D_\n윤민호,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,광주광역시,합계,NaN,NaN,NaN,1172429.0,694252.0,573995,34487,40916,33312,0,0,0,0,0,682710,11542.0,478177.0
2,NaN,광주광역시,광주광역시,동구,계,NaN,82265.0,51550.0,42139,3489,2688,2101,0,0,0,0,0,50417,1133.0,30715.0
3,NaN,광주광역시,광주광역시,서구,계,NaN,247275.0,147798.0,122803,7163,8813,6765,0,0,0,0,0,145544,2254.0,99477.0
4,NaN,광주광역시,광주광역시,남구,계,NaN,176964.0,106294.0,88699,5569,6042,4143,0,0,0,0,0,104453,1841.0,70670.0
5,NaN,광주광역시,광주광역시,북구,계,NaN,358914.0,210587.0,173766,9854,11694,11693,0,0,0,0,0,207007,3580.0,148327.0
6,NaN,광주광역시,광주광역시,광산구,계,NaN,307011.0,178023.0,146588,8412,11679,8610,0,0,0,0,0,175289,2734.0,128988.0


In [41]:
gwangju_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [42]:
rename_gwangju = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_이용섭',
    'Unnamed: 9': '득표수_3_바른미래당_전덕영',
    'Unnamed: 10': '득표수_5_정의당_나경채',
    'Unnamed: 11': '득표수_6_민중당_윤민호',
    'Unnamed: 17': '득표수_계'
}

In [43]:
gwangju_7th = gwangju_7th.rename(columns=rename_gwangju).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
gwangju_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이용섭,득표수_3_바른미래당_전덕영,득표수_5_정의당_나경채,득표수_6_민중당_윤민호,득표수_계,무효투표수,기권수
1,합계,NaN,1172429.0,694252.0,573995,34487,40916,33312,682710,11542.0,478177.0
2,광주광역시,동구,82265.0,51550.0,42139,3489,2688,2101,50417,1133.0,30715.0
3,광주광역시,서구,247275.0,147798.0,122803,7163,8813,6765,145544,2254.0,99477.0
4,광주광역시,남구,176964.0,106294.0,88699,5569,6042,4143,104453,1841.0,70670.0
5,광주광역시,북구,358914.0,210587.0,173766,9854,11694,11693,207007,3580.0,148327.0
6,광주광역시,광산구,307011.0,178023.0,146588,8412,11679,8610,175289,2734.0,128988.0


In [44]:
gwangju_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 1 to 6
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                6 non-null      object 
 1   구시군               5 non-null      object 
 2   선거인수              6 non-null      float64
 3   투표수               6 non-null      float64
 4   득표수_1_더불어민주당_이용섭  6 non-null      object 
 5   득표수_3_바른미래당_전덕영   6 non-null      object 
 6   득표수_5_정의당_나경채     6 non-null      object 
 7   득표수_6_민중당_윤민호     6 non-null      object 
 8   득표수_계             6 non-null      object 
 9   무효투표수             6 non-null      float64
 10  기권수               6 non-null      float64
dtypes: float64(4), object(7)
memory usage: 660.0+ bytes


In [45]:
gwangju_7th = gwangju_7th.astype({col: int for col in gwangju_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gwangju_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이용섭,득표수_3_바른미래당_전덕영,득표수_5_정의당_나경채,득표수_6_민중당_윤민호,득표수_계,무효투표수,기권수
1,합계,NaN,1172429,694252,573995,34487,40916,33312,682710,11542,478177
2,광주광역시,동구,82265,51550,42139,3489,2688,2101,50417,1133,30715
3,광주광역시,서구,247275,147798,122803,7163,8813,6765,145544,2254,99477
4,광주광역시,남구,176964,106294,88699,5569,6042,4143,104453,1841,70670
5,광주광역시,북구,358914,210587,173766,9854,11694,11693,207007,3580,148327
6,광주광역시,광산구,307011,178023,146588,8412,11679,8610,175289,2734,128988


In [46]:
gwangju_7th_with_total = gwangju_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '광주광역시'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [47]:
gwangju_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이용섭,득표수_3_바른미래당_전덕영,득표수_5_정의당_나경채,득표수_6_민중당_윤민호,득표수_계,무효투표수,기권수
1,광주광역시,합계,1172429,694252,573995,34487,40916,33312,682710,11542,478177
2,광주광역시,동구,82265,51550,42139,3489,2688,2101,50417,1133,30715
3,광주광역시,서구,247275,147798,122803,7163,8813,6765,145544,2254,99477
4,광주광역시,남구,176964,106294,88699,5569,6042,4143,104453,1841,70670
5,광주광역시,북구,358914,210587,173766,9854,11694,11693,207007,3580,148327
6,광주광역시,광산구,307011,178023,146588,8412,11679,8610,175289,2734,128988


In [48]:
gwangju_7th_with_total.to_csv("temp1_governor_gwangju_7.csv", index=False, encoding="utf-8-sig")


## Daejeon


In [49]:
daejeon_7th = dfs_7th['대전광역시']
daejeon_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,대전광역시,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n허태정,자유한국당_x000D_\n박성효,바른미래당_x000D_\n남충희,정의당_x000D_\n김윤기,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,대전광역시,합계,NaN,NaN,NaN,1219513.0,706983.0,393354,224306,61271,18351,0,0,0,0,0,697282,9701.0,512530.0
2,NaN,대전광역시,대전광역시,동구,계,NaN,191362.0,106341.0,55928,35880,9750,2755,0,0,0,0,0,104313,2028.0,85021.0
3,NaN,대전광역시,대전광역시,중구,계,NaN,206042.0,116950.0,60878,41632,9863,2730,0,0,0,0,0,115103,1847.0,89092.0
4,NaN,대전광역시,대전광역시,서구,계,NaN,395367.0,226532.0,127151,71325,19262,6140,0,0,0,0,0,223878,2654.0,168835.0
5,NaN,대전광역시,대전광역시,유성구,계,NaN,272663.0,168310.0,101687,43981,16466,4571,0,0,0,0,0,166705,1605.0,104353.0
6,NaN,대전광역시,대전광역시,대덕구,계,NaN,154079.0,88850.0,47710,31488,5930,2155,0,0,0,0,0,87283,1567.0,65229.0


In [50]:
daejeon_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [51]:
rename_daejeon = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_허태정',
    'Unnamed: 9': '득표수_2_자유한국당_박성효',
    'Unnamed: 10': '득표수_3_바른미래당_남충희',
    'Unnamed: 11': '득표수_5_정의당_김윤기',
    'Unnamed: 17': '득표수_계'
}

In [52]:
daejeon_7th = daejeon_7th.rename(columns=rename_daejeon).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
daejeon_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_허태정,득표수_2_자유한국당_박성효,득표수_3_바른미래당_남충희,득표수_5_정의당_김윤기,득표수_계,무효투표수,기권수
1,합계,NaN,1219513.0,706983.0,393354,224306,61271,18351,697282,9701.0,512530.0
2,대전광역시,동구,191362.0,106341.0,55928,35880,9750,2755,104313,2028.0,85021.0
3,대전광역시,중구,206042.0,116950.0,60878,41632,9863,2730,115103,1847.0,89092.0
4,대전광역시,서구,395367.0,226532.0,127151,71325,19262,6140,223878,2654.0,168835.0
5,대전광역시,유성구,272663.0,168310.0,101687,43981,16466,4571,166705,1605.0,104353.0
6,대전광역시,대덕구,154079.0,88850.0,47710,31488,5930,2155,87283,1567.0,65229.0


In [53]:
daejeon_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 1 to 6
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                6 non-null      object 
 1   구시군               5 non-null      object 
 2   선거인수              6 non-null      float64
 3   투표수               6 non-null      float64
 4   득표수_1_더불어민주당_허태정  6 non-null      object 
 5   득표수_2_자유한국당_박성효   6 non-null      object 
 6   득표수_3_바른미래당_남충희   6 non-null      object 
 7   득표수_5_정의당_김윤기     6 non-null      object 
 8   득표수_계             6 non-null      object 
 9   무효투표수             6 non-null      float64
 10  기권수               6 non-null      float64
dtypes: float64(4), object(7)
memory usage: 660.0+ bytes


In [54]:
daejeon_7th = daejeon_7th.astype({col: int for col in daejeon_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
daejeon_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_허태정,득표수_2_자유한국당_박성효,득표수_3_바른미래당_남충희,득표수_5_정의당_김윤기,득표수_계,무효투표수,기권수
1,합계,NaN,1219513,706983,393354,224306,61271,18351,697282,9701,512530
2,대전광역시,동구,191362,106341,55928,35880,9750,2755,104313,2028,85021
3,대전광역시,중구,206042,116950,60878,41632,9863,2730,115103,1847,89092
4,대전광역시,서구,395367,226532,127151,71325,19262,6140,223878,2654,168835
5,대전광역시,유성구,272663,168310,101687,43981,16466,4571,166705,1605,104353
6,대전광역시,대덕구,154079,88850,47710,31488,5930,2155,87283,1567,65229


In [55]:
daejeon_7th_with_total = daejeon_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '대전광역시'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)


In [56]:
daejeon_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_허태정,득표수_2_자유한국당_박성효,득표수_3_바른미래당_남충희,득표수_5_정의당_김윤기,득표수_계,무효투표수,기권수
1,대전광역시,합계,1219513,706983,393354,224306,61271,18351,697282,9701,512530
2,대전광역시,동구,191362,106341,55928,35880,9750,2755,104313,2028,85021
3,대전광역시,중구,206042,116950,60878,41632,9863,2730,115103,1847,89092
4,대전광역시,서구,395367,226532,127151,71325,19262,6140,223878,2654,168835
5,대전광역시,유성구,272663,168310,101687,43981,16466,4571,166705,1605,104353
6,대전광역시,대덕구,154079,88850,47710,31488,5930,2155,87283,1567,65229


In [57]:
daejeon_7th_with_total.to_csv("temp1_governor_daejeon_7.csv", index=False, encoding="utf-8-sig")


## Ulsan


In [58]:
ulsan_7th = dfs_7th['울산광역시']
ulsan_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,울산광역시,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n송철호,자유한국당_x000D_\n김기현,바른미래당_x000D_\n이영희,민중당_x000D_\n김창현,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,울산광역시,합계,NaN,NaN,NaN,942550.0,610698.0,317341,240475,13589,28621,0,0,0,0,0,600026,10672.0,331852.0
2,NaN,울산광역시,울산광역시,중구,계,NaN,194908.0,128337.0,65014,54227,2553,4078,0,0,0,0,0,125872,2465.0,66571.0
3,NaN,울산광역시,울산광역시,남구,계,NaN,274930.0,176694.0,89518,75240,3502,5989,0,0,0,0,0,174249,2445.0,98236.0
4,NaN,울산광역시,울산광역시,동구,계,NaN,134316.0,86763.0,47190,28418,2151,7154,0,0,0,0,0,84913,1850.0,47553.0
5,NaN,울산광역시,울산광역시,북구,계,NaN,159030.0,104132.0,58639,33758,2506,7943,0,0,0,0,0,102846,1286.0,54898.0
6,NaN,울산광역시,울산광역시,울주군,계,NaN,179366.0,114772.0,56980,48832,2877,3457,0,0,0,0,0,112146,2626.0,64594.0


In [59]:
ulsan_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [60]:
rename_ulsan = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_송철호',
    'Unnamed: 9': '득표수_2_자유한국당_김기현',
    'Unnamed: 10': '득표수_3_바른미래당_이영희',
    'Unnamed: 11': '득표수_6_민중당_김창현',
    'Unnamed: 17': '득표수_계'
}

In [61]:
ulsan_7th = ulsan_7th.rename(columns=rename_ulsan).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
ulsan_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송철호,득표수_2_자유한국당_김기현,득표수_3_바른미래당_이영희,득표수_6_민중당_김창현,득표수_계,무효투표수,기권수
1,합계,NaN,942550.0,610698.0,317341,240475,13589,28621,600026,10672.0,331852.0
2,울산광역시,중구,194908.0,128337.0,65014,54227,2553,4078,125872,2465.0,66571.0
3,울산광역시,남구,274930.0,176694.0,89518,75240,3502,5989,174249,2445.0,98236.0
4,울산광역시,동구,134316.0,86763.0,47190,28418,2151,7154,84913,1850.0,47553.0
5,울산광역시,북구,159030.0,104132.0,58639,33758,2506,7943,102846,1286.0,54898.0
6,울산광역시,울주군,179366.0,114772.0,56980,48832,2877,3457,112146,2626.0,64594.0


In [62]:
ulsan_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 1 to 6
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                6 non-null      object 
 1   구시군               5 non-null      object 
 2   선거인수              6 non-null      float64
 3   투표수               6 non-null      float64
 4   득표수_1_더불어민주당_송철호  6 non-null      object 
 5   득표수_2_자유한국당_김기현   6 non-null      object 
 6   득표수_3_바른미래당_이영희   6 non-null      object 
 7   득표수_6_민중당_김창현     6 non-null      object 
 8   득표수_계             6 non-null      object 
 9   무효투표수             6 non-null      float64
 10  기권수               6 non-null      float64
dtypes: float64(4), object(7)
memory usage: 660.0+ bytes


In [63]:
ulsan_7th = ulsan_7th.astype({col: int for col in ulsan_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
ulsan_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송철호,득표수_2_자유한국당_김기현,득표수_3_바른미래당_이영희,득표수_6_민중당_김창현,득표수_계,무효투표수,기권수
1,합계,NaN,942550,610698,317341,240475,13589,28621,600026,10672,331852
2,울산광역시,중구,194908,128337,65014,54227,2553,4078,125872,2465,66571
3,울산광역시,남구,274930,176694,89518,75240,3502,5989,174249,2445,98236
4,울산광역시,동구,134316,86763,47190,28418,2151,7154,84913,1850,47553
5,울산광역시,북구,159030,104132,58639,33758,2506,7943,102846,1286,54898
6,울산광역시,울주군,179366,114772,56980,48832,2877,3457,112146,2626,64594


In [64]:
ulsan_7th_with_total = ulsan_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '울산광역시'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)


In [65]:
ulsan_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송철호,득표수_2_자유한국당_김기현,득표수_3_바른미래당_이영희,득표수_6_민중당_김창현,득표수_계,무효투표수,기권수
1,울산광역시,합계,942550,610698,317341,240475,13589,28621,600026,10672,331852
2,울산광역시,중구,194908,128337,65014,54227,2553,4078,125872,2465,66571
3,울산광역시,남구,274930,176694,89518,75240,3502,5989,174249,2445,98236
4,울산광역시,동구,134316,86763,47190,28418,2151,7154,84913,1850,47553
5,울산광역시,북구,159030,104132,58639,33758,2506,7943,102846,1286,54898
6,울산광역시,울주군,179366,114772,56980,48832,2877,3457,112146,2626,64594


In [66]:
ulsan_7th_with_total.to_csv("temp1_governor_ulsan_7.csv", index=False, encoding="utf-8-sig")


## Sejong


In [67]:
sejong_7th = dfs_7th['세종특별자치시']
sejong_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,세종특별자치시,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n이춘희,자유한국당_x000D_\n송아영,바른미래당_x000D_\n허철회,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,세종특별자치시,합계,NaN,NaN,NaN,222852.0,137603.0,96896,24546,14444,0,0,0,0,0,0,135886,1717.0,85249.0
2,NaN,세종특별자치시,세종특별자치시,세종특별자치시,계,NaN,222852.0,137603.0,96896,24546,14444,0,0,0,0,0,0,135886,1717.0,85249.0


In [68]:
sejong_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [69]:
rename_sejong = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_이춘희',
    'Unnamed: 9': '득표수_2_자유한국당_송아영',
    'Unnamed: 10': '득표수_3_바른미래당_허철회',
    'Unnamed: 17': '득표수_계'
}

In [70]:
sejong_7th = sejong_7th.rename(columns=rename_sejong).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
sejong_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이춘희,득표수_2_자유한국당_송아영,득표수_3_바른미래당_허철회,득표수_계,무효투표수,기권수
1,합계,NaN,222852.0,137603.0,96896,24546,14444,135886,1717.0,85249.0
2,세종특별자치시,세종특별자치시,222852.0,137603.0,96896,24546,14444,135886,1717.0,85249.0


In [71]:
sejong_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 1 to 2
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                2 non-null      object 
 1   구시군               1 non-null      object 
 2   선거인수              2 non-null      float64
 3   투표수               2 non-null      float64
 4   득표수_1_더불어민주당_이춘희  2 non-null      object 
 5   득표수_2_자유한국당_송아영   2 non-null      object 
 6   득표수_3_바른미래당_허철회   2 non-null      object 
 7   득표수_계             2 non-null      object 
 8   무효투표수             2 non-null      float64
 9   기권수               2 non-null      float64
dtypes: float64(4), object(6)
memory usage: 292.0+ bytes


In [72]:
sejong_7th = sejong_7th.astype({col: int for col in sejong_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
sejong_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이춘희,득표수_2_자유한국당_송아영,득표수_3_바른미래당_허철회,득표수_계,무효투표수,기권수
1,합계,NaN,222852,137603,96896,24546,14444,135886,1717,85249
2,세종특별자치시,세종특별자치시,222852,137603,96896,24546,14444,135886,1717,85249


In [73]:
sejong_7th_with_total = sejong_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '세종특별자치시'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
).drop(index=2)

In [74]:
sejong_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이춘희,득표수_2_자유한국당_송아영,득표수_3_바른미래당_허철회,득표수_계,무효투표수,기권수
1,세종특별자치시,합계,222852,137603,96896,24546,14444,135886,1717,85249


In [75]:
sejong_7th_with_total.to_csv("temp1_governor_sejong_7.csv", index=False, encoding="utf-8-sig")


## Gyeonggi


In [76]:
gyeonggi_7th = dfs_7th['경기도']
gyeonggi_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,경기도,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n이재명,자유한국당_x000D_\n남경필,바른미래당_x000D_\n김영환,정의당_x000D_\n이홍우,민중당_x000D_\n홍성규,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,경기도,합계,NaN,NaN,NaN,10533027.0,6084955.0,3370621,2122433,287504,151871,43098,0,0,0,0,5975527,109428.0,4448072.0
2,NaN,경기도,경기도,수원시장안구,계,NaN,202140.0,121951.0,64965,45405,5638,3050,768,0,0,0,0,119826,2125.0,80189.0
3,NaN,경기도,경기도,수원시권선구,계,NaN,342897.0,197363.0,110066,69133,8656,4787,1473,0,0,0,0,194115,3248.0,145534.0
4,NaN,경기도,경기도,수원시팔달구,계,NaN,161673.0,88388.0,44464,36057,3707,1908,552,0,0,0,0,86688,1700.0,73285.0
5,NaN,경기도,경기도,수원시영통구,계,NaN,273531.0,174692.0,98513,59730,8134,4762,873,0,0,0,0,172012,2680.0,98839.0
6,NaN,경기도,경기도,성남시수정구,계,NaN,204053.0,115704.0,71400,34928,4592,1902,822,0,0,0,0,113644,2060.0,88349.0
7,NaN,경기도,경기도,성남시중원구,계,NaN,195192.0,112200.0,70848,32910,4179,1535,916,0,0,0,0,110388,1812.0,82992.0
8,NaN,경기도,경기도,성남시분당구,계,NaN,401672.0,266966.0,141379,102110,13286,5498,989,0,0,0,0,263262,3704.0,134706.0
9,NaN,경기도,경기도,의정부시,계,NaN,366459.0,198518.0,111187,69513,8276,4593,1408,0,0,0,0,194977,3541.0,167941.0


In [77]:
gyeonggi_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [78]:
rename_gyeonggi = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_이재명',
    'Unnamed: 9': '득표수_2_자유한국당_남경필',
    'Unnamed: 10': '득표수_3_바른미래당_김영환',
    'Unnamed: 11': '득표수_5_정의당_이홍우',
    'Unnamed: 12': '득표수_6_민중당_홍성규',
    'Unnamed: 17': '득표수_계'
}

In [79]:
gyeonggi_7th = gyeonggi_7th.rename(columns=rename_gyeonggi).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
gyeonggi_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_자유한국당_남경필,득표수_3_바른미래당_김영환,득표수_5_정의당_이홍우,득표수_6_민중당_홍성규,득표수_계,무효투표수,기권수
1,합계,NaN,10533027.0,6084955.0,3370621,2122433,287504,151871,43098,5975527,109428.0,4448072.0
2,경기도,수원시장안구,202140.0,121951.0,64965,45405,5638,3050,768,119826,2125.0,80189.0
3,경기도,수원시권선구,342897.0,197363.0,110066,69133,8656,4787,1473,194115,3248.0,145534.0
4,경기도,수원시팔달구,161673.0,88388.0,44464,36057,3707,1908,552,86688,1700.0,73285.0
5,경기도,수원시영통구,273531.0,174692.0,98513,59730,8134,4762,873,172012,2680.0,98839.0
6,경기도,성남시수정구,204053.0,115704.0,71400,34928,4592,1902,822,113644,2060.0,88349.0
7,경기도,성남시중원구,195192.0,112200.0,70848,32910,4179,1535,916,110388,1812.0,82992.0
8,경기도,성남시분당구,401672.0,266966.0,141379,102110,13286,5498,989,263262,3704.0,134706.0
9,경기도,의정부시,366459.0,198518.0,111187,69513,8276,4593,1408,194977,3541.0,167941.0
10,경기도,안양시만안구,216498.0,129861.0,70458,47791,5517,3027,728,127521,2340.0,86637.0


In [80]:
gyeonggi_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 1 to 43
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                43 non-null     object 
 1   구시군               42 non-null     object 
 2   선거인수              43 non-null     float64
 3   투표수               43 non-null     float64
 4   득표수_1_더불어민주당_이재명  43 non-null     object 
 5   득표수_2_자유한국당_남경필   43 non-null     object 
 6   득표수_3_바른미래당_김영환   43 non-null     object 
 7   득표수_5_정의당_이홍우     43 non-null     object 
 8   득표수_6_민중당_홍성규     43 non-null     object 
 9   득표수_계             43 non-null     object 
 10  무효투표수             43 non-null     float64
 11  기권수               43 non-null     float64
dtypes: float64(4), object(8)
memory usage: 4.2+ KB


In [81]:
gyeonggi_7th = gyeonggi_7th.astype({col: int for col in gyeonggi_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gyeonggi_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_자유한국당_남경필,득표수_3_바른미래당_김영환,득표수_5_정의당_이홍우,득표수_6_민중당_홍성규,득표수_계,무효투표수,기권수
1,합계,NaN,10533027,6084955,3370621,2122433,287504,151871,43098,5975527,109428,4448072
2,경기도,수원시장안구,202140,121951,64965,45405,5638,3050,768,119826,2125,80189
3,경기도,수원시권선구,342897,197363,110066,69133,8656,4787,1473,194115,3248,145534
4,경기도,수원시팔달구,161673,88388,44464,36057,3707,1908,552,86688,1700,73285
5,경기도,수원시영통구,273531,174692,98513,59730,8134,4762,873,172012,2680,98839
6,경기도,성남시수정구,204053,115704,71400,34928,4592,1902,822,113644,2060,88349
7,경기도,성남시중원구,195192,112200,70848,32910,4179,1535,916,110388,1812,82992
8,경기도,성남시분당구,401672,266966,141379,102110,13286,5498,989,263262,3704,134706
9,경기도,의정부시,366459,198518,111187,69513,8276,4593,1408,194977,3541,167941
10,경기도,안양시만안구,216498,129861,70458,47791,5517,3027,728,127521,2340,86637


In [82]:
gyeonggi_7th_with_total = gyeonggi_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '경기도'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [83]:
gyeonggi_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_자유한국당_남경필,득표수_3_바른미래당_김영환,득표수_5_정의당_이홍우,득표수_6_민중당_홍성규,득표수_계,무효투표수,기권수
1,경기도,합계,10533027,6084955,3370621,2122433,287504,151871,43098,5975527,109428,4448072
2,경기도,수원시장안구,202140,121951,64965,45405,5638,3050,768,119826,2125,80189
3,경기도,수원시권선구,342897,197363,110066,69133,8656,4787,1473,194115,3248,145534
4,경기도,수원시팔달구,161673,88388,44464,36057,3707,1908,552,86688,1700,73285
5,경기도,수원시영통구,273531,174692,98513,59730,8134,4762,873,172012,2680,98839
6,경기도,성남시수정구,204053,115704,71400,34928,4592,1902,822,113644,2060,88349
7,경기도,성남시중원구,195192,112200,70848,32910,4179,1535,916,110388,1812,82992
8,경기도,성남시분당구,401672,266966,141379,102110,13286,5498,989,263262,3704,134706
9,경기도,의정부시,366459,198518,111187,69513,8276,4593,1408,194977,3541,167941
10,경기도,안양시만안구,216498,129861,70458,47791,5517,3027,728,127521,2340,86637


In [84]:
gyeonggi_7th_with_total.to_csv("temp1_governor_gyeonggi_7.csv", index=False, encoding="utf-8-sig")


## Gangwon


In [85]:
gangwon_7th = dfs_7th['강원도']
gangwon_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,강원도,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n최문순,자유한국당_x000D_\n정창수,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,강원도,합계,NaN,NaN,NaN,1296196.0,819193.0,518447,282456,0,0,0,0,0,0,0,800903,18290.0,477003.0
2,NaN,강원도,강원도,춘천시,계,NaN,231430.0,144282.0,96699,45024,0,0,0,0,0,0,0,141723,2559.0,87148.0
3,NaN,강원도,강원도,원주시,계,NaN,278991.0,159235.0,103811,53060,0,0,0,0,0,0,0,156871,2364.0,119756.0
4,NaN,강원도,강원도,강릉시,계,NaN,179767.0,108772.0,62434,44086,0,0,0,0,0,0,0,106520,2252.0,70995.0
5,NaN,강원도,강원도,동해시,계,NaN,75747.0,46923.0,29034,16783,0,0,0,0,0,0,0,45817,1106.0,28824.0
6,NaN,강원도,강원도,삼척시,계,NaN,59117.0,41032.0,25401,14369,0,0,0,0,0,0,0,39770,1262.0,18085.0
7,NaN,강원도,강원도,태백시,계,NaN,38386.0,25298.0,15354,9272,0,0,0,0,0,0,0,24626,672.0,13088.0
8,NaN,강원도,강원도,정선군,계,NaN,33293.0,23092.0,14708,7739,0,0,0,0,0,0,0,22447,645.0,10201.0
9,NaN,강원도,강원도,속초시,계,NaN,68234.0,40493.0,26237,13443,0,0,0,0,0,0,0,39680,813.0,27741.0


In [86]:
gangwon_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [87]:
rename_gangwon = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_최문순',
    'Unnamed: 9': '득표수_2_자유한국당_정창수',
    'Unnamed: 17': '득표수_계'
}

In [88]:
gangwon_7th = gangwon_7th.rename(columns=rename_gangwon).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
gangwon_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_최문순,득표수_2_자유한국당_정창수,득표수_계,무효투표수,기권수
1,합계,NaN,1296196.0,819193.0,518447,282456,800903,18290.0,477003.0
2,강원도,춘천시,231430.0,144282.0,96699,45024,141723,2559.0,87148.0
3,강원도,원주시,278991.0,159235.0,103811,53060,156871,2364.0,119756.0
4,강원도,강릉시,179767.0,108772.0,62434,44086,106520,2252.0,70995.0
5,강원도,동해시,75747.0,46923.0,29034,16783,45817,1106.0,28824.0
6,강원도,삼척시,59117.0,41032.0,25401,14369,39770,1262.0,18085.0
7,강원도,태백시,38386.0,25298.0,15354,9272,24626,672.0,13088.0
8,강원도,정선군,33293.0,23092.0,14708,7739,22447,645.0,10201.0
9,강원도,속초시,68234.0,40493.0,26237,13443,39680,813.0,27741.0
10,강원도,고성군,25496.0,18354.0,11620,6173,17793,561.0,7142.0


In [89]:
gangwon_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 1 to 19
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                19 non-null     object 
 1   구시군               18 non-null     object 
 2   선거인수              19 non-null     float64
 3   투표수               19 non-null     float64
 4   득표수_1_더불어민주당_최문순  19 non-null     object 
 5   득표수_2_자유한국당_정창수   19 non-null     object 
 6   득표수_계             19 non-null     object 
 7   무효투표수             19 non-null     float64
 8   기권수               19 non-null     float64
dtypes: float64(4), object(5)
memory usage: 1.5+ KB


In [90]:
gangwon_7th = gangwon_7th.astype({col: int for col in gangwon_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gangwon_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_최문순,득표수_2_자유한국당_정창수,득표수_계,무효투표수,기권수
1,합계,NaN,1296196,819193,518447,282456,800903,18290,477003
2,강원도,춘천시,231430,144282,96699,45024,141723,2559,87148
3,강원도,원주시,278991,159235,103811,53060,156871,2364,119756
4,강원도,강릉시,179767,108772,62434,44086,106520,2252,70995
5,강원도,동해시,75747,46923,29034,16783,45817,1106,28824
6,강원도,삼척시,59117,41032,25401,14369,39770,1262,18085
7,강원도,태백시,38386,25298,15354,9272,24626,672,13088
8,강원도,정선군,33293,23092,14708,7739,22447,645,10201
9,강원도,속초시,68234,40493,26237,13443,39680,813,27741
10,강원도,고성군,25496,18354,11620,6173,17793,561,7142


In [91]:
gangwon_7th_with_total = gangwon_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '강원도'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [92]:
gangwon_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_최문순,득표수_2_자유한국당_정창수,득표수_계,무효투표수,기권수
1,강원도,합계,1296196,819193,518447,282456,800903,18290,477003
2,강원도,춘천시,231430,144282,96699,45024,141723,2559,87148
3,강원도,원주시,278991,159235,103811,53060,156871,2364,119756
4,강원도,강릉시,179767,108772,62434,44086,106520,2252,70995
5,강원도,동해시,75747,46923,29034,16783,45817,1106,28824
6,강원도,삼척시,59117,41032,25401,14369,39770,1262,18085
7,강원도,태백시,38386,25298,15354,9272,24626,672,13088
8,강원도,정선군,33293,23092,14708,7739,22447,645,10201
9,강원도,속초시,68234,40493,26237,13443,39680,813,27741
10,강원도,고성군,25496,18354,11620,6173,17793,561,7142


In [93]:
gangwon_7th_with_total.to_csv("temp1_governor_gangwon_7.csv", index=False, encoding="utf-8-sig")


## Chungbuk


In [94]:
chungbuk_7th = dfs_7th['충청북도']
chungbuk_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,충청북도,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n이시종,자유한국당_x000D_\n박경국,바른미래당_x000D_\n신용한,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,충청북도,합계,NaN,NaN,NaN,1318186.0,782316.0,468750,227371,70330,0,0,0,0,0,0,766451,15865.0,535870.0
2,NaN,충청북도,충청북도,청주시상당구,계,NaN,140405.0,80162.0,47260,23920,7676,0,0,0,0,0,0,78856,1306.0,60243.0
3,NaN,충청북도,충청북도,청주시서원구,계,NaN,174307.0,100218.0,60096,28299,10359,0,0,0,0,0,0,98754,1464.0,74089.0
4,NaN,충청북도,충청북도,청주시흥덕구,계,NaN,204639.0,110108.0,67695,28114,12684,0,0,0,0,0,0,108493,1615.0,94531.0
5,NaN,충청북도,충청북도,청주시청원구,계,NaN,154249.0,85634.0,54139,21312,8988,0,0,0,0,0,0,84439,1195.0,68615.0
6,NaN,충청북도,충청북도,충주시,계,NaN,175414.0,103606.0,62569,31253,7887,0,0,0,0,0,0,101709,1897.0,71808.0
7,NaN,충청북도,충청북도,제천시,계,NaN,114513.0,70289.0,41702,21580,5428,0,0,0,0,0,0,68710,1579.0,44224.0
8,NaN,충청북도,충청북도,단양군,계,NaN,26796.0,19078.0,10661,6807,1070,0,0,0,0,0,0,18538,540.0,7718.0
9,NaN,충청북도,충청북도,영동군,계,NaN,43904.0,31226.0,17317,10871,1979,0,0,0,0,0,0,30167,1059.0,12678.0


In [95]:
chungbuk_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [96]:
rename_chungbuk = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_이시종',
    'Unnamed: 9': '득표수_2_자유한국당_박경국',
    'Unnamed: 10': '득표수_3_바른미래당_신용한',
    'Unnamed: 17': '득표수_계'
}

In [97]:
chungbuk_7th = chungbuk_7th.rename(columns=rename_chungbuk).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
chungbuk_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이시종,득표수_2_자유한국당_박경국,득표수_3_바른미래당_신용한,득표수_계,무효투표수,기권수
1,합계,NaN,1318186.0,782316.0,468750,227371,70330,766451,15865.0,535870.0
2,충청북도,청주시상당구,140405.0,80162.0,47260,23920,7676,78856,1306.0,60243.0
3,충청북도,청주시서원구,174307.0,100218.0,60096,28299,10359,98754,1464.0,74089.0
4,충청북도,청주시흥덕구,204639.0,110108.0,67695,28114,12684,108493,1615.0,94531.0
5,충청북도,청주시청원구,154249.0,85634.0,54139,21312,8988,84439,1195.0,68615.0
6,충청북도,충주시,175414.0,103606.0,62569,31253,7887,101709,1897.0,71808.0
7,충청북도,제천시,114513.0,70289.0,41702,21580,5428,68710,1579.0,44224.0
8,충청북도,단양군,26796.0,19078.0,10661,6807,1070,18538,540.0,7718.0
9,충청북도,영동군,43904.0,31226.0,17317,10871,1979,30167,1059.0,12678.0
10,충청북도,보은군,30016.0,22735.0,10627,9856,1391,21874,861.0,7281.0


In [98]:
chungbuk_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 1 to 15
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                15 non-null     object 
 1   구시군               14 non-null     object 
 2   선거인수              15 non-null     float64
 3   투표수               15 non-null     float64
 4   득표수_1_더불어민주당_이시종  15 non-null     object 
 5   득표수_2_자유한국당_박경국   15 non-null     object 
 6   득표수_3_바른미래당_신용한   15 non-null     object 
 7   득표수_계             15 non-null     object 
 8   무효투표수             15 non-null     float64
 9   기권수               15 non-null     float64
dtypes: float64(4), object(6)
memory usage: 1.3+ KB


In [99]:
chungbuk_7th = chungbuk_7th.astype({col: int for col in chungbuk_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
chungbuk_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이시종,득표수_2_자유한국당_박경국,득표수_3_바른미래당_신용한,득표수_계,무효투표수,기권수
1,합계,NaN,1318186,782316,468750,227371,70330,766451,15865,535870
2,충청북도,청주시상당구,140405,80162,47260,23920,7676,78856,1306,60243
3,충청북도,청주시서원구,174307,100218,60096,28299,10359,98754,1464,74089
4,충청북도,청주시흥덕구,204639,110108,67695,28114,12684,108493,1615,94531
5,충청북도,청주시청원구,154249,85634,54139,21312,8988,84439,1195,68615
6,충청북도,충주시,175414,103606,62569,31253,7887,101709,1897,71808
7,충청북도,제천시,114513,70289,41702,21580,5428,68710,1579,44224
8,충청북도,단양군,26796,19078,10661,6807,1070,18538,540,7718
9,충청북도,영동군,43904,31226,17317,10871,1979,30167,1059,12678
10,충청북도,보은군,30016,22735,10627,9856,1391,21874,861,7281


In [100]:
chungbuk_7th_with_total = chungbuk_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '충청북도'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [101]:
chungbuk_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이시종,득표수_2_자유한국당_박경국,득표수_3_바른미래당_신용한,득표수_계,무효투표수,기권수
1,충청북도,합계,1318186,782316,468750,227371,70330,766451,15865,535870
2,충청북도,청주시상당구,140405,80162,47260,23920,7676,78856,1306,60243
3,충청북도,청주시서원구,174307,100218,60096,28299,10359,98754,1464,74089
4,충청북도,청주시흥덕구,204639,110108,67695,28114,12684,108493,1615,94531
5,충청북도,청주시청원구,154249,85634,54139,21312,8988,84439,1195,68615
6,충청북도,충주시,175414,103606,62569,31253,7887,101709,1897,71808
7,충청북도,제천시,114513,70289,41702,21580,5428,68710,1579,44224
8,충청북도,단양군,26796,19078,10661,6807,1070,18538,540,7718
9,충청북도,영동군,43904,31226,17317,10871,1979,30167,1059,12678
10,충청북도,보은군,30016,22735,10627,9856,1391,21874,861,7281


In [102]:
chungbuk_7th_with_total.to_csv("temp1_governor_chungbuk_7.csv", index=False, encoding="utf-8-sig")


## Chungnam


In [103]:
chungnam_7th = dfs_7th['충청남도']
chungnam_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,충청남도,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n양승조,자유한국당_x000D_\n이인제,코리아_x000D_\n차국환,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,충청남도,합계,NaN,NaN,NaN,1740413.0,1011447.0,615870,345577,23012,0,0,0,0,0,0,984459,26988.0,728966.0
2,NaN,충청남도,충청남도,천안시서북구,계,NaN,298093.0,159458.0,117098,37427,2705,0,0,0,0,0,0,157230,2228.0,138635.0
3,NaN,충청남도,충청남도,천안시동남구,계,NaN,211724.0,106417.0,69999,32706,1773,0,0,0,0,0,0,104478,1939.0,105307.0
4,NaN,충청남도,충청남도,공주시,계,NaN,92935.0,56756.0,32526,21453,1128,0,0,0,0,0,0,55107,1649.0,36179.0
5,NaN,충청남도,충청남도,보령시,계,NaN,87281.0,56352.0,28981,23856,1384,0,0,0,0,0,0,54221,2131.0,30929.0
6,NaN,충청남도,충청남도,아산시,계,NaN,244293.0,129980.0,84978,40171,2498,0,0,0,0,0,0,127647,2333.0,114313.0
7,NaN,충청남도,충청남도,서산시,계,NaN,139529.0,82003.0,49800,28417,1662,0,0,0,0,0,0,79879,2124.0,57526.0
8,NaN,충청남도,충청남도,태안군,계,NaN,55792.0,37240.0,19534,14832,1146,0,0,0,0,0,0,35512,1728.0,18552.0
9,NaN,충청남도,충청남도,금산군,계,NaN,46310.0,30788.0,16748,11711,1199,0,0,0,0,0,0,29658,1130.0,15522.0


In [104]:
chungnam_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [105]:
rename_chungnam = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_양승조',
    'Unnamed: 9': '득표수_2_자유한국당_이인제',
    'Unnamed: 10': '득표수_6_가자코리아_차국환',
    'Unnamed: 17': '득표수_계'
}

In [106]:
chungnam_7th = chungnam_7th.rename(columns=rename_chungnam).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
chungnam_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양승조,득표수_2_자유한국당_이인제,득표수_6_가자코리아_차국환,득표수_계,무효투표수,기권수
1,합계,NaN,1740413.0,1011447.0,615870,345577,23012,984459,26988.0,728966.0
2,충청남도,천안시서북구,298093.0,159458.0,117098,37427,2705,157230,2228.0,138635.0
3,충청남도,천안시동남구,211724.0,106417.0,69999,32706,1773,104478,1939.0,105307.0
4,충청남도,공주시,92935.0,56756.0,32526,21453,1128,55107,1649.0,36179.0
5,충청남도,보령시,87281.0,56352.0,28981,23856,1384,54221,2131.0,30929.0
6,충청남도,아산시,244293.0,129980.0,84978,40171,2498,127647,2333.0,114313.0
7,충청남도,서산시,139529.0,82003.0,49800,28417,1662,79879,2124.0,57526.0
8,충청남도,태안군,55792.0,37240.0,19534,14832,1146,35512,1728.0,18552.0
9,충청남도,금산군,46310.0,30788.0,16748,11711,1199,29658,1130.0,15522.0
10,충청남도,논산시,103367.0,62960.0,35905,23871,1251,61027,1933.0,40407.0


In [107]:
chungnam_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 1 to 17
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                17 non-null     object 
 1   구시군               16 non-null     object 
 2   선거인수              17 non-null     float64
 3   투표수               17 non-null     float64
 4   득표수_1_더불어민주당_양승조  17 non-null     object 
 5   득표수_2_자유한국당_이인제   17 non-null     object 
 6   득표수_6_가자코리아_차국환   17 non-null     object 
 7   득표수_계             17 non-null     object 
 8   무효투표수             17 non-null     float64
 9   기권수               17 non-null     float64
dtypes: float64(4), object(6)
memory usage: 1.5+ KB


In [108]:
chungnam_7th = chungnam_7th.astype({col: int for col in chungnam_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
chungnam_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양승조,득표수_2_자유한국당_이인제,득표수_6_가자코리아_차국환,득표수_계,무효투표수,기권수
1,합계,NaN,1740413,1011447,615870,345577,23012,984459,26988,728966
2,충청남도,천안시서북구,298093,159458,117098,37427,2705,157230,2228,138635
3,충청남도,천안시동남구,211724,106417,69999,32706,1773,104478,1939,105307
4,충청남도,공주시,92935,56756,32526,21453,1128,55107,1649,36179
5,충청남도,보령시,87281,56352,28981,23856,1384,54221,2131,30929
6,충청남도,아산시,244293,129980,84978,40171,2498,127647,2333,114313
7,충청남도,서산시,139529,82003,49800,28417,1662,79879,2124,57526
8,충청남도,태안군,55792,37240,19534,14832,1146,35512,1728,18552
9,충청남도,금산군,46310,30788,16748,11711,1199,29658,1130,15522
10,충청남도,논산시,103367,62960,35905,23871,1251,61027,1933,40407


In [109]:
chungnam_7th_with_total = chungnam_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '충청남도'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [110]:
chungnam_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양승조,득표수_2_자유한국당_이인제,득표수_6_가자코리아_차국환,득표수_계,무효투표수,기권수
1,충청남도,합계,1740413,1011447,615870,345577,23012,984459,26988,728966
2,충청남도,천안시서북구,298093,159458,117098,37427,2705,157230,2228,138635
3,충청남도,천안시동남구,211724,106417,69999,32706,1773,104478,1939,105307
4,충청남도,공주시,92935,56756,32526,21453,1128,55107,1649,36179
5,충청남도,보령시,87281,56352,28981,23856,1384,54221,2131,30929
6,충청남도,아산시,244293,129980,84978,40171,2498,127647,2333,114313
7,충청남도,서산시,139529,82003,49800,28417,1662,79879,2124,57526
8,충청남도,태안군,55792,37240,19534,14832,1146,35512,1728,18552
9,충청남도,금산군,46310,30788,16748,11711,1199,29658,1130,15522
10,충청남도,논산시,103367,62960,35905,23871,1251,61027,1933,40407


In [111]:
chungnam_7th_with_total.to_csv("temp1_governor_chungnam_7.csv", index=False, encoding="utf-8-sig")


## Jeonbuk


In [112]:
jeonbuk_7th = dfs_7th['전라북도']
jeonbuk_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,전라북도,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n송하진,자유한국당_x000D_\n신재봉,민주평화당_x000D_\n임정엽,정의당_x000D_\n권태홍,민중당_x000D_\n이광석,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,전라북도,합계,NaN,NaN,NaN,1527729.0,996504.0,682042,26374,184728,52496,20827,0,0,0,0,966467,30037.0,531225.0
2,NaN,전라북도,전라북도,전주시완산구,계,NaN,313929.0,193303.0,121422,4205,48517,13719,2092,0,0,0,0,189955,3348.0,120626.0
3,NaN,전라북도,전라북도,전주시덕진구,계,NaN,210274.0,130716.0,80531,2713,34008,9616,1721,0,0,0,0,128589,2127.0,79558.0
4,NaN,전라북도,전라북도,군산시,계,NaN,223434.0,134447.0,98640,4475,17447,6656,3205,0,0,0,0,130423,4024.0,88987.0
5,NaN,전라북도,전라북도,익산시,계,NaN,245566.0,153275.0,111862,3844,22976,7986,2244,0,0,0,0,148912,4363.0,92291.0
6,NaN,전라북도,전라북도,정읍시,계,NaN,96002.0,64334.0,47427,1468,8792,2642,1413,0,0,0,0,61742,2592.0,31668.0
7,NaN,전라북도,전라북도,남원시,계,NaN,69930.0,51530.0,37718,1532,7153,1687,1462,0,0,0,0,49552,1978.0,18400.0
8,NaN,전라북도,전라북도,김제시,계,NaN,74992.0,51664.0,37736,1242,8258,1585,873,0,0,0,0,49694,1970.0,23328.0
9,NaN,전라북도,전라북도,완주군,계,NaN,78531.0,52837.0,31420,1206,14548,2791,1210,0,0,0,0,51175,1662.0,25694.0


In [113]:
jeonbuk_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [114]:
rename_jeonbuk = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_송하진',
    'Unnamed: 9': '득표수_2_자유한국당_신재봉',
    'Unnamed: 10': '득표수_4_민주평화당_임정엽',
    'Unnamed: 11': '득표수_5_정의당_권태홍',
    'Unnamed: 12': '득표수_6_민중당_이광석',
    'Unnamed: 17': '득표수_계'
}

In [115]:
jeonbuk_7th = jeonbuk_7th.rename(columns=rename_jeonbuk).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
jeonbuk_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송하진,득표수_2_자유한국당_신재봉,득표수_4_민주평화당_임정엽,득표수_5_정의당_권태홍,득표수_6_민중당_이광석,득표수_계,무효투표수,기권수
1,합계,NaN,1527729.0,996504.0,682042,26374,184728,52496,20827,966467,30037.0,531225.0
2,전라북도,전주시완산구,313929.0,193303.0,121422,4205,48517,13719,2092,189955,3348.0,120626.0
3,전라북도,전주시덕진구,210274.0,130716.0,80531,2713,34008,9616,1721,128589,2127.0,79558.0
4,전라북도,군산시,223434.0,134447.0,98640,4475,17447,6656,3205,130423,4024.0,88987.0
5,전라북도,익산시,245566.0,153275.0,111862,3844,22976,7986,2244,148912,4363.0,92291.0
6,전라북도,정읍시,96002.0,64334.0,47427,1468,8792,2642,1413,61742,2592.0,31668.0
7,전라북도,남원시,69930.0,51530.0,37718,1532,7153,1687,1462,49552,1978.0,18400.0
8,전라북도,김제시,74992.0,51664.0,37736,1242,8258,1585,873,49694,1970.0,23328.0
9,전라북도,완주군,78531.0,52837.0,31420,1206,14548,2791,1210,51175,1662.0,25694.0
10,전라북도,진안군,23029.0,17585.0,11593,913,2916,827,578,16827,758.0,5444.0


In [116]:
jeonbuk_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 1 to 16
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                16 non-null     object 
 1   구시군               15 non-null     object 
 2   선거인수              16 non-null     float64
 3   투표수               16 non-null     float64
 4   득표수_1_더불어민주당_송하진  16 non-null     object 
 5   득표수_2_자유한국당_신재봉   16 non-null     object 
 6   득표수_4_민주평화당_임정엽   16 non-null     object 
 7   득표수_5_정의당_권태홍     16 non-null     object 
 8   득표수_6_민중당_이광석     16 non-null     object 
 9   득표수_계             16 non-null     object 
 10  무효투표수             16 non-null     float64
 11  기권수               16 non-null     float64
dtypes: float64(4), object(8)
memory usage: 1.6+ KB


In [117]:
jeonbuk_7th = jeonbuk_7th.astype({col: int for col in jeonbuk_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
jeonbuk_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송하진,득표수_2_자유한국당_신재봉,득표수_4_민주평화당_임정엽,득표수_5_정의당_권태홍,득표수_6_민중당_이광석,득표수_계,무효투표수,기권수
1,합계,NaN,1527729,996504,682042,26374,184728,52496,20827,966467,30037,531225
2,전라북도,전주시완산구,313929,193303,121422,4205,48517,13719,2092,189955,3348,120626
3,전라북도,전주시덕진구,210274,130716,80531,2713,34008,9616,1721,128589,2127,79558
4,전라북도,군산시,223434,134447,98640,4475,17447,6656,3205,130423,4024,88987
5,전라북도,익산시,245566,153275,111862,3844,22976,7986,2244,148912,4363,92291
6,전라북도,정읍시,96002,64334,47427,1468,8792,2642,1413,61742,2592,31668
7,전라북도,남원시,69930,51530,37718,1532,7153,1687,1462,49552,1978,18400
8,전라북도,김제시,74992,51664,37736,1242,8258,1585,873,49694,1970,23328
9,전라북도,완주군,78531,52837,31420,1206,14548,2791,1210,51175,1662,25694
10,전라북도,진안군,23029,17585,11593,913,2916,827,578,16827,758,5444


In [118]:
jeonbuk_7th_with_total = jeonbuk_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '전라북도'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [119]:
jeonbuk_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송하진,득표수_2_자유한국당_신재봉,득표수_4_민주평화당_임정엽,득표수_5_정의당_권태홍,득표수_6_민중당_이광석,득표수_계,무효투표수,기권수
1,전라북도,합계,1527729,996504,682042,26374,184728,52496,20827,966467,30037,531225
2,전라북도,전주시완산구,313929,193303,121422,4205,48517,13719,2092,189955,3348,120626
3,전라북도,전주시덕진구,210274,130716,80531,2713,34008,9616,1721,128589,2127,79558
4,전라북도,군산시,223434,134447,98640,4475,17447,6656,3205,130423,4024,88987
5,전라북도,익산시,245566,153275,111862,3844,22976,7986,2244,148912,4363,92291
6,전라북도,정읍시,96002,64334,47427,1468,8792,2642,1413,61742,2592,31668
7,전라북도,남원시,69930,51530,37718,1532,7153,1687,1462,49552,1978,18400
8,전라북도,김제시,74992,51664,37736,1242,8258,1585,873,49694,1970,23328
9,전라북도,완주군,78531,52837,31420,1206,14548,2791,1210,51175,1662,25694
10,전라북도,진안군,23029,17585,11593,913,2916,827,578,16827,758,5444


In [120]:
jeonbuk_7th_with_total.to_csv("temp1_governor_jeonbuk_7.csv", index=False, encoding="utf-8-sig")


## Jeonnam


In [121]:
jeonnam_7th = dfs_7th['전라남도']
jeonnam_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,전라남도,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n김영록,바른미래당_x000D_\n박매호,민주평화당_x000D_\n민영삼,정의당_x000D_\n노형태,민중당_x000D_\n이성수,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,전라남도,합계,NaN,NaN,NaN,1577224.0,1092107.0,807902,40287,110973,37433,51410,0,0,0,0,1048005,44102.0,485117.0
2,NaN,전라남도,전라남도,목포시,계,NaN,186864.0,119950.0,87419,2644,18579,5724,2399,0,0,0,0,116765,3185.0,66914.0
3,NaN,전라남도,전라남도,여수시,계,NaN,235359.0,153554.0,120941,6198,10972,4184,7401,0,0,0,0,149696,3858.0,81805.0
4,NaN,전라남도,전라남도,순천시,계,NaN,225673.0,144341.0,113180,4942,9399,4603,8612,0,0,0,0,140736,3605.0,81332.0
5,NaN,전라남도,전라남도,나주시,계,NaN,93513.0,61444.0,47100,2317,4753,2141,2800,0,0,0,0,59111,2333.0,32069.0
6,NaN,전라남도,전라남도,광양시,계,NaN,122154.0,83720.0,63778,4210,5466,3290,4351,0,0,0,0,81095,2625.0,38434.0
7,NaN,전라남도,전라남도,담양군,계,NaN,41703.0,29739.0,22127,1361,2098,1181,1772,0,0,0,0,28539,1200.0,11964.0
8,NaN,전라남도,전라남도,장성군,계,NaN,39603.0,29751.0,20820,1294,2453,1156,2350,0,0,0,0,28073,1678.0,9852.0
9,NaN,전라남도,전라남도,곡성군,계,NaN,26739.0,20723.0,13989,1219,2058,970,1280,0,0,0,0,19516,1207.0,6016.0


In [122]:
jeonnam_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [123]:
rename_jeonnam = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_김영록',
    'Unnamed: 9': '득표수_3_바른미래당_박매호',
    'Unnamed: 10': '득표수_4_민주평화당_민영삼',
    'Unnamed: 11': '득표수_5_정의당_노형태',
    'Unnamed: 12': '득표수_6_민중당_이성수',
    'Unnamed: 17': '득표수_계'
}

In [124]:
jeonnam_7th = jeonnam_7th.rename(columns=rename_jeonnam).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
jeonnam_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김영록,득표수_3_바른미래당_박매호,득표수_4_민주평화당_민영삼,득표수_5_정의당_노형태,득표수_6_민중당_이성수,득표수_계,무효투표수,기권수
1,합계,NaN,1577224.0,1092107.0,807902,40287,110973,37433,51410,1048005,44102.0,485117.0
2,전라남도,목포시,186864.0,119950.0,87419,2644,18579,5724,2399,116765,3185.0,66914.0
3,전라남도,여수시,235359.0,153554.0,120941,6198,10972,4184,7401,149696,3858.0,81805.0
4,전라남도,순천시,225673.0,144341.0,113180,4942,9399,4603,8612,140736,3605.0,81332.0
5,전라남도,나주시,93513.0,61444.0,47100,2317,4753,2141,2800,59111,2333.0,32069.0
6,전라남도,광양시,122154.0,83720.0,63778,4210,5466,3290,4351,81095,2625.0,38434.0
7,전라남도,담양군,41703.0,29739.0,22127,1361,2098,1181,1772,28539,1200.0,11964.0
8,전라남도,장성군,39603.0,29751.0,20820,1294,2453,1156,2350,28073,1678.0,9852.0
9,전라남도,곡성군,26739.0,20723.0,13989,1219,2058,970,1280,19516,1207.0,6016.0
10,전라남도,구례군,23743.0,19333.0,13080,723,2236,839,1396,18274,1059.0,4410.0


In [125]:
jeonnam_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 1 to 23
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                23 non-null     object 
 1   구시군               22 non-null     object 
 2   선거인수              23 non-null     float64
 3   투표수               23 non-null     float64
 4   득표수_1_더불어민주당_김영록  23 non-null     object 
 5   득표수_3_바른미래당_박매호   23 non-null     object 
 6   득표수_4_민주평화당_민영삼   23 non-null     object 
 7   득표수_5_정의당_노형태     23 non-null     object 
 8   득표수_6_민중당_이성수     23 non-null     object 
 9   득표수_계             23 non-null     object 
 10  무효투표수             23 non-null     float64
 11  기권수               23 non-null     float64
dtypes: float64(4), object(8)
memory usage: 2.3+ KB


In [126]:
jeonnam_7th = jeonnam_7th.astype({col: int for col in jeonnam_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
jeonnam_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김영록,득표수_3_바른미래당_박매호,득표수_4_민주평화당_민영삼,득표수_5_정의당_노형태,득표수_6_민중당_이성수,득표수_계,무효투표수,기권수
1,합계,NaN,1577224,1092107,807902,40287,110973,37433,51410,1048005,44102,485117
2,전라남도,목포시,186864,119950,87419,2644,18579,5724,2399,116765,3185,66914
3,전라남도,여수시,235359,153554,120941,6198,10972,4184,7401,149696,3858,81805
4,전라남도,순천시,225673,144341,113180,4942,9399,4603,8612,140736,3605,81332
5,전라남도,나주시,93513,61444,47100,2317,4753,2141,2800,59111,2333,32069
6,전라남도,광양시,122154,83720,63778,4210,5466,3290,4351,81095,2625,38434
7,전라남도,담양군,41703,29739,22127,1361,2098,1181,1772,28539,1200,11964
8,전라남도,장성군,39603,29751,20820,1294,2453,1156,2350,28073,1678,9852
9,전라남도,곡성군,26739,20723,13989,1219,2058,970,1280,19516,1207,6016
10,전라남도,구례군,23743,19333,13080,723,2236,839,1396,18274,1059,4410


In [127]:
jeonnam_7th_with_total = jeonnam_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '전라남도'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [128]:
jeonnam_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김영록,득표수_3_바른미래당_박매호,득표수_4_민주평화당_민영삼,득표수_5_정의당_노형태,득표수_6_민중당_이성수,득표수_계,무효투표수,기권수
1,전라남도,합계,1577224,1092107,807902,40287,110973,37433,51410,1048005,44102,485117
2,전라남도,목포시,186864,119950,87419,2644,18579,5724,2399,116765,3185,66914
3,전라남도,여수시,235359,153554,120941,6198,10972,4184,7401,149696,3858,81805
4,전라남도,순천시,225673,144341,113180,4942,9399,4603,8612,140736,3605,81332
5,전라남도,나주시,93513,61444,47100,2317,4753,2141,2800,59111,2333,32069
6,전라남도,광양시,122154,83720,63778,4210,5466,3290,4351,81095,2625,38434
7,전라남도,담양군,41703,29739,22127,1361,2098,1181,1772,28539,1200,11964
8,전라남도,장성군,39603,29751,20820,1294,2453,1156,2350,28073,1678,9852
9,전라남도,곡성군,26739,20723,13989,1219,2058,970,1280,19516,1207,6016
10,전라남도,구례군,23743,19333,13080,723,2236,839,1396,18274,1059,4410


In [129]:
jeonnam_7th_with_total.to_csv("temp1_governor_jeonnam_7.csv", index=False, encoding="utf-8-sig")


## Gyeongbuk


In [130]:
gyeongbuk_7th = dfs_7th['경상북도']
gyeongbuk_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,경상북도,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n오중기,자유한국당_x000D_\n이철우,바른미래당_x000D_\n권오을,정의당_x000D_\n박창호,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,경상북도,합계,NaN,NaN,NaN,2251538.0,1457539.0,482564,732785,143409,47250,0,0,0,0,0,1406008,51531.0,793999.0
2,NaN,경상북도,경상북도,포항시북구,계,NaN,223110.0,139955.0,57440,64206,10354,4719,0,0,0,0,0,136719,3236.0,83155.0
3,NaN,경상북도,경상북도,포항시남구,계,NaN,198696.0,119400.0,49535,54509,8995,3564,0,0,0,0,0,116603,2797.0,79296.0
4,NaN,경상북도,경상북도,울릉군,계,NaN,9057.0,7479.0,2087,4222,580,255,0,0,0,0,0,7144,335.0,1578.0
5,NaN,경상북도,경상북도,경주시,계,NaN,218273.0,139142.0,44294,73050,11842,5111,0,0,0,0,0,134297,4845.0,79131.0
6,NaN,경상북도,경상북도,김천시,계,NaN,119511.0,83567.0,26500,45278,6606,2379,0,0,0,0,0,80763,2804.0,35944.0
7,NaN,경상북도,경상북도,안동시,계,NaN,137423.0,94434.0,28292,40002,20806,2146,0,0,0,0,0,91246,3188.0,42989.0
8,NaN,경상북도,경상북도,구미시,계,NaN,332873.0,187236.0,78892,82603,16234,5474,0,0,0,0,0,183203,4033.0,145637.0
9,NaN,경상북도,경상북도,영주시,계,NaN,91447.0,63733.0,19849,31646,8040,1825,0,0,0,0,0,61360,2373.0,27714.0


In [131]:
gyeongbuk_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [132]:
rename_gyeongbuk = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_오중기',
    'Unnamed: 9': '득표수_2_자유한국당_이철우',
    'Unnamed: 10': '득표수_3_바른미래당_권오을',
    'Unnamed: 11': '득표수_5_정의당_박창호',
    'Unnamed: 17': '득표수_계'
}

In [133]:
gyeongbuk_7th = gyeongbuk_7th.rename(columns=rename_gyeongbuk).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
gyeongbuk_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오중기,득표수_2_자유한국당_이철우,득표수_3_바른미래당_권오을,득표수_5_정의당_박창호,득표수_계,무효투표수,기권수
1,합계,NaN,2251538.0,1457539.0,482564,732785,143409,47250,1406008,51531.0,793999.0
2,경상북도,포항시북구,223110.0,139955.0,57440,64206,10354,4719,136719,3236.0,83155.0
3,경상북도,포항시남구,198696.0,119400.0,49535,54509,8995,3564,116603,2797.0,79296.0
4,경상북도,울릉군,9057.0,7479.0,2087,4222,580,255,7144,335.0,1578.0
5,경상북도,경주시,218273.0,139142.0,44294,73050,11842,5111,134297,4845.0,79131.0
6,경상북도,김천시,119511.0,83567.0,26500,45278,6606,2379,80763,2804.0,35944.0
7,경상북도,안동시,137423.0,94434.0,28292,40002,20806,2146,91246,3188.0,42989.0
8,경상북도,구미시,332873.0,187236.0,78892,82603,16234,5474,183203,4033.0,145637.0
9,경상북도,영주시,91447.0,63733.0,19849,31646,8040,1825,61360,2373.0,27714.0
10,경상북도,영천시,88000.0,60058.0,16867,32770,5494,2289,57420,2638.0,27942.0


In [134]:
gyeongbuk_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 1 to 25
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                25 non-null     object 
 1   구시군               24 non-null     object 
 2   선거인수              25 non-null     float64
 3   투표수               25 non-null     float64
 4   득표수_1_더불어민주당_오중기  25 non-null     object 
 5   득표수_2_자유한국당_이철우   25 non-null     object 
 6   득표수_3_바른미래당_권오을   25 non-null     object 
 7   득표수_5_정의당_박창호     25 non-null     object 
 8   득표수_계             25 non-null     object 
 9   무효투표수             25 non-null     float64
 10  기권수               25 non-null     float64
dtypes: float64(4), object(7)
memory usage: 2.3+ KB


In [135]:
gyeongbuk_7th = gyeongbuk_7th.astype({col: int for col in gyeongbuk_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gyeongbuk_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오중기,득표수_2_자유한국당_이철우,득표수_3_바른미래당_권오을,득표수_5_정의당_박창호,득표수_계,무효투표수,기권수
1,합계,NaN,2251538,1457539,482564,732785,143409,47250,1406008,51531,793999
2,경상북도,포항시북구,223110,139955,57440,64206,10354,4719,136719,3236,83155
3,경상북도,포항시남구,198696,119400,49535,54509,8995,3564,116603,2797,79296
4,경상북도,울릉군,9057,7479,2087,4222,580,255,7144,335,1578
5,경상북도,경주시,218273,139142,44294,73050,11842,5111,134297,4845,79131
6,경상북도,김천시,119511,83567,26500,45278,6606,2379,80763,2804,35944
7,경상북도,안동시,137423,94434,28292,40002,20806,2146,91246,3188,42989
8,경상북도,구미시,332873,187236,78892,82603,16234,5474,183203,4033,145637
9,경상북도,영주시,91447,63733,19849,31646,8040,1825,61360,2373,27714
10,경상북도,영천시,88000,60058,16867,32770,5494,2289,57420,2638,27942


In [136]:
gyeongbuk_7th_with_total = gyeongbuk_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '경상북도'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [137]:
gyeongbuk_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오중기,득표수_2_자유한국당_이철우,득표수_3_바른미래당_권오을,득표수_5_정의당_박창호,득표수_계,무효투표수,기권수
1,경상북도,합계,2251538,1457539,482564,732785,143409,47250,1406008,51531,793999
2,경상북도,포항시북구,223110,139955,57440,64206,10354,4719,136719,3236,83155
3,경상북도,포항시남구,198696,119400,49535,54509,8995,3564,116603,2797,79296
4,경상북도,울릉군,9057,7479,2087,4222,580,255,7144,335,1578
5,경상북도,경주시,218273,139142,44294,73050,11842,5111,134297,4845,79131
6,경상북도,김천시,119511,83567,26500,45278,6606,2379,80763,2804,35944
7,경상북도,안동시,137423,94434,28292,40002,20806,2146,91246,3188,42989
8,경상북도,구미시,332873,187236,78892,82603,16234,5474,183203,4033,145637
9,경상북도,영주시,91447,63733,19849,31646,8040,1825,61360,2373,27714
10,경상북도,영천시,88000,60058,16867,32770,5494,2289,57420,2638,27942


In [138]:
gyeongbuk_7th_with_total.to_csv("temp1_governor_gyeongbuk_7.csv", index=False, encoding="utf-8-sig")


## Gyeongnam


In [139]:
gyeongnam_7th = dfs_7th['경상남도']
gyeongnam_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,경상남도,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n김경수,자유한국당_x000D_\n김태호,바른미래당_x000D_\n김유근,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,경상남도,합계,NaN,NaN,NaN,2765485.0,1819391.0,941491,765809,75418,0,0,0,0,0,0,1782718,36673.0,946094.0
2,NaN,경상남도,경상남도,창원시의창구,계,NaN,210327.0,135147.0,72745,54066,6264,0,0,0,0,0,0,133075,2072.0,75180.0
3,NaN,경상남도,경상남도,창원시성산구,계,NaN,184600.0,125797.0,76104,42016,6012,0,0,0,0,0,0,124132,1665.0,58803.0
4,NaN,경상남도,경상남도,창원시마산합포구,계,NaN,154311.0,99281.0,44305,48933,4348,0,0,0,0,0,0,97586,1695.0,55030.0
5,NaN,경상남도,경상남도,창원시마산회원구,계,NaN,166500.0,109609.0,52918,50373,4628,0,0,0,0,0,0,107919,1690.0,56891.0
6,NaN,경상남도,경상남도,창원시진해구,계,NaN,150396.0,93901.0,50350,37660,4381,0,0,0,0,0,0,92391,1510.0,56495.0
7,NaN,경상남도,경상남도,진주시,계,NaN,282049.0,190842.0,96104,83637,8004,0,0,0,0,0,0,187745,3097.0,91207.0
8,NaN,경상남도,경상남도,통영시,계,NaN,110317.0,72815.0,32916,35501,2890,0,0,0,0,0,0,71307,1508.0,37502.0
9,NaN,경상남도,경상남도,고성군,계,NaN,46588.0,34075.0,16355,15438,1091,0,0,0,0,0,0,32884,1191.0,12513.0


In [140]:
gyeongnam_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [141]:
rename_gyeongnam = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_김경수',
    'Unnamed: 9': '득표수_2_자유한국당_김태호',
    'Unnamed: 10': '득표수_3_바른미래당_김유근',
    'Unnamed: 17': '득표수_계'
}

In [142]:
gyeongnam_7th = gyeongnam_7th.rename(columns=rename_gyeongnam).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
gyeongnam_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김경수,득표수_2_자유한국당_김태호,득표수_3_바른미래당_김유근,득표수_계,무효투표수,기권수
1,합계,NaN,2765485.0,1819391.0,941491,765809,75418,1782718,36673.0,946094.0
2,경상남도,창원시의창구,210327.0,135147.0,72745,54066,6264,133075,2072.0,75180.0
3,경상남도,창원시성산구,184600.0,125797.0,76104,42016,6012,124132,1665.0,58803.0
4,경상남도,창원시마산합포구,154311.0,99281.0,44305,48933,4348,97586,1695.0,55030.0
5,경상남도,창원시마산회원구,166500.0,109609.0,52918,50373,4628,107919,1690.0,56891.0
6,경상남도,창원시진해구,150396.0,93901.0,50350,37660,4381,92391,1510.0,56495.0
7,경상남도,진주시,282049.0,190842.0,96104,83637,8004,187745,3097.0,91207.0
8,경상남도,통영시,110317.0,72815.0,32916,35501,2890,71307,1508.0,37502.0
9,경상남도,고성군,46588.0,34075.0,16355,15438,1091,32884,1191.0,12513.0
10,경상남도,사천시,94449.0,65481.0,29390,31787,2599,63776,1705.0,28968.0


In [143]:
gyeongnam_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 1 to 23
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                23 non-null     object 
 1   구시군               22 non-null     object 
 2   선거인수              23 non-null     float64
 3   투표수               23 non-null     float64
 4   득표수_1_더불어민주당_김경수  23 non-null     object 
 5   득표수_2_자유한국당_김태호   23 non-null     object 
 6   득표수_3_바른미래당_김유근   23 non-null     object 
 7   득표수_계             23 non-null     object 
 8   무효투표수             23 non-null     float64
 9   기권수               23 non-null     float64
dtypes: float64(4), object(6)
memory usage: 1.9+ KB


In [144]:
gyeongnam_7th = gyeongnam_7th.astype({col: int for col in gyeongnam_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gyeongnam_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김경수,득표수_2_자유한국당_김태호,득표수_3_바른미래당_김유근,득표수_계,무효투표수,기권수
1,합계,NaN,2765485,1819391,941491,765809,75418,1782718,36673,946094
2,경상남도,창원시의창구,210327,135147,72745,54066,6264,133075,2072,75180
3,경상남도,창원시성산구,184600,125797,76104,42016,6012,124132,1665,58803
4,경상남도,창원시마산합포구,154311,99281,44305,48933,4348,97586,1695,55030
5,경상남도,창원시마산회원구,166500,109609,52918,50373,4628,107919,1690,56891
6,경상남도,창원시진해구,150396,93901,50350,37660,4381,92391,1510,56495
7,경상남도,진주시,282049,190842,96104,83637,8004,187745,3097,91207
8,경상남도,통영시,110317,72815,32916,35501,2890,71307,1508,37502
9,경상남도,고성군,46588,34075,16355,15438,1091,32884,1191,12513
10,경상남도,사천시,94449,65481,29390,31787,2599,63776,1705,28968


In [145]:
gyeongnam_7th_with_total = gyeongnam_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '경상남도'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [146]:
gyeongnam_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김경수,득표수_2_자유한국당_김태호,득표수_3_바른미래당_김유근,득표수_계,무효투표수,기권수
1,경상남도,합계,2765485,1819391,941491,765809,75418,1782718,36673,946094
2,경상남도,창원시의창구,210327,135147,72745,54066,6264,133075,2072,75180
3,경상남도,창원시성산구,184600,125797,76104,42016,6012,124132,1665,58803
4,경상남도,창원시마산합포구,154311,99281,44305,48933,4348,97586,1695,55030
5,경상남도,창원시마산회원구,166500,109609,52918,50373,4628,107919,1690,56891
6,경상남도,창원시진해구,150396,93901,50350,37660,4381,92391,1510,56495
7,경상남도,진주시,282049,190842,96104,83637,8004,187745,3097,91207
8,경상남도,통영시,110317,72815,32916,35501,2890,71307,1508,37502
9,경상남도,고성군,46588,34075,16355,15438,1091,32884,1191,12513
10,경상남도,사천시,94449,65481,29390,31787,2599,63776,1705,28968


In [147]:
gyeongnam_7th_with_total.to_csv("temp1_governor_gyeongnam_7.csv", index=False, encoding="utf-8-sig")


## Jeju

In [148]:
jeju_7th = dfs_7th['제주특별자치도']
jeju_7th

,선거종류,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,무효투표수,기권수
0,시·도지사선거,제주특별자치도,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n문대림,자유한국당_x000D_\n김방훈,바른미래당_x000D_\n장성철,녹색당_x000D_\n고은영,무소속_x000D_\n원희룡,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,제주특별자치도,합계,NaN,NaN,NaN,532515.0,350943.0,137901,11241,5019,12188,178255,0,0,0,0,344604,6339.0,181572.0
2,NaN,제주특별자치도,제주특별자치도,제주시,계,NaN,383917.0,250176.0,97935,8785,3874,8860,126594,0,0,0,0,246048,4128.0,133741.0
3,NaN,제주특별자치도,제주특별자치도,서귀포시,계,NaN,148598.0,100767.0,39966,2456,1145,3328,51661,0,0,0,0,98556,2211.0,47831.0


In [149]:
jeju_7th.columns.tolist()

['선거종류',
 '선거구명',
 '시도명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 '무효투표수',
 '기권수']

In [150]:
rename_jeju = {
    '시도명': '시도',
    '구시군명': '구시군',
    '후보자별 득표수': '득표수_1_더불어민주당_문대림',
    'Unnamed: 9': '득표수_2_자유한국당_김방훈',
    'Unnamed: 10': '득표수_3_바른미래당_장성철',
    'Unnamed: 11': '득표수_6_녹색당_고은영',
    'Unnamed: 12': '득표수_7_무소속_원희룡',
    'Unnamed: 17': '득표수_계'
}

In [151]:
jeju_7th = jeju_7th.rename(columns=rename_jeju).drop(columns=['선거종류', '선거구명', '읍면동명', '구분', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']).drop(index=0)
jeju_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_문대림,득표수_2_자유한국당_김방훈,득표수_3_바른미래당_장성철,득표수_6_녹색당_고은영,득표수_7_무소속_원희룡,득표수_계,무효투표수,기권수
1,합계,NaN,532515.0,350943.0,137901,11241,5019,12188,178255,344604,6339.0,181572.0
2,제주특별자치도,제주시,383917.0,250176.0,97935,8785,3874,8860,126594,246048,4128.0,133741.0
3,제주특별자치도,서귀포시,148598.0,100767.0,39966,2456,1145,3328,51661,98556,2211.0,47831.0


In [152]:
jeju_7th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 1 to 3
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   시도                3 non-null      object 
 1   구시군               2 non-null      object 
 2   선거인수              3 non-null      float64
 3   투표수               3 non-null      float64
 4   득표수_1_더불어민주당_문대림  3 non-null      object 
 5   득표수_2_자유한국당_김방훈   3 non-null      object 
 6   득표수_3_바른미래당_장성철   3 non-null      object 
 7   득표수_6_녹색당_고은영     3 non-null      object 
 8   득표수_7_무소속_원희룡     3 non-null      object 
 9   득표수_계             3 non-null      object 
 10  무효투표수             3 non-null      float64
 11  기권수               3 non-null      float64
dtypes: float64(4), object(8)
memory usage: 420.0+ bytes


In [153]:
jeju_7th = jeju_7th.astype({col: int for col in jeju_7th.select_dtypes(include='float').columns}).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
jeju_7th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_문대림,득표수_2_자유한국당_김방훈,득표수_3_바른미래당_장성철,득표수_6_녹색당_고은영,득표수_7_무소속_원희룡,득표수_계,무효투표수,기권수
1,합계,NaN,532515,350943,137901,11241,5019,12188,178255,344604,6339,181572
2,제주특별자치도,제주시,383917,250176,97935,8785,3874,8860,126594,246048,4128,133741
3,제주특별자치도,서귀포시,148598,100767,39966,2456,1145,3328,51661,98556,2211,47831


In [154]:
jeju_7th_with_total = jeju_7th.assign(
    시도=lambda df: df['시도'].mask(df.index == df.index[0], '제주특별자치도'),
    구시군=lambda df: df['구시군'].mask(df.index == df.index[0], '합계')
)

In [155]:
jeju_7th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_문대림,득표수_2_자유한국당_김방훈,득표수_3_바른미래당_장성철,득표수_6_녹색당_고은영,득표수_7_무소속_원희룡,득표수_계,무효투표수,기권수
1,제주특별자치도,합계,532515,350943,137901,11241,5019,12188,178255,344604,6339,181572
2,제주특별자치도,제주시,383917,250176,97935,8785,3874,8860,126594,246048,4128,133741
3,제주특별자치도,서귀포시,148598,100767,39966,2456,1145,3328,51661,98556,2211,47831


In [156]:
jeju_7th_with_total.to_csv("temp1_governor_jeju_7.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [157]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp1_governor_ulsan_7.csv
Added: temp1_governor_jeonbuk_7.csv
Added: temp1_governor_gyeongnam_7.csv
Added: temp1_governor_gwangju_7.csv
Added: temp1_governor_daejeon_7.csv
Added: temp1_governor_gangwon_7.csv
Added: temp1_governor_gyeonggi_7.csv
Added: temp1_governor_jeju_7.csv
Added: temp1_governor_daegu_7.csv
Added: temp1_governor_sejong_7.csv
Added: temp1_governor_seoul_7.csv
Added: temp1_governor_gyeongbuk_7.csv
Added: temp1_governor_incheon_7.csv
Added: temp1_governor_busan_7.csv
Added: temp1_governor_chungbuk_7.csv
Added: temp1_governor_chungnam_7.csv
Added: temp1_governor_jeonnam_7.csv
Total 17 files compressed.
